# For light/dark experiments - Collect all the data that has been tracked and associated


In [1]:
import numpy as np
import os
import json
import sys
import glob
import time
import datetime
from pympler.tracker import SummaryTracker
from scipy import stats, signal
from scipy.signal import argrelextrema
from sklearn import linear_model
import cv2
import h5py
from itertools import groupby
from math import ceil
import pickle
import math
from scipy.linalg import norm
from scipy.spatial import ConvexHull
from scipy.signal import butter, lfilter
from sklearn.decomposition import PCA
import random
import feather


# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from matplotlib.patches import Rectangle, Circle
from matplotlib.patches import Polygon, Ellipse
from matplotlib import patches
from matplotlib.collections import PatchCollection
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D

# import statsmodels.api as sm

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/AntTrackCode')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker

# import multiprocessing
# import threading
import subprocess
from subprocess import call
import pandas as pd
import seaborn as sns
import copy


# stop telling me i have a nan in an array with a logical comparison
with np.errstate(invalid='ignore'):
    np.less([np.nan, 0], 1)

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


In [2]:
%qtconsole

## Build file list 

In [2]:
vid_locations = '/media/gravishlab/SeagateExpansionDrive/Light_Dark/'
file_list = []

# searches for files
file_list = glob.glob(os.path.join(vid_locations, 'Tunnel_2019**/**/*0000.mp4'))
# file_list = glob.glob(os.path.join(vid_locations, 'Tunnel_20180329-30/**/*0000.mp4'))
file_list = sorted(file_list)
print('Total Number of Videos: ',len(file_list))
                
file_list = file_list
len(file_list)

Total Number of Videos:  3903


3903

### Calibrations 

In [66]:
# TUNNEL EXPERIMENTS
pix2mm = 31.72 # Measured 3cm in 4 cameras
fps = 239.16

v_multiplier=  fps / pix2mm
print('%0.3f pixels per mm'%pix2mm)


hei = 550
wid = 1000

pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']

31.720 pixels per mm


# LOAD IN DATA FROM PICKLE - if have already been saved as done in next section

In [4]:
# LOAD IN DATA FROM PICKLE OR FEATHER (SKIP ALL NEXT DATAFRAME SECTIONS) - SPECIFY WHETHER TO INCLUDE TD DATA
print('loading in all trackways pickle')
start_time = time.time()
colonies = glob.glob(os.path.join(vid_locations, 'Tunnel_2019**/'))
for col in colonies[0:]:
    col_OI = col.split('/')[-2][-5:]
    print('   colony: %s'%col_OI)
    tmp = pd.read_pickle(vid_locations + 'AllTracks_' + col_OI)
#     tmp = pd.read_pickle(vid_locations + 'AllTracks_TDs_' + col_OI)
    if col == colonies[0]:
        df = tmp.copy()
    else:
        df = pd.concat([df,tmp], ignore_index=True)
    del tmp, col_OI
print('read in all trackways pickle -- duration: ', time.time()-start_time)


# remove trials with really weird velocity values
df = df.drop(df[df['v'].apply(lambda x: np.nanmax(x))>2000].index)
df = df.drop(df[df['vfilt'].apply(lambda x: np.nanmax(x))>2000].index)
print('\nRemoved trackways with a weirdly high velocity value')


# load in master dataframe
# print('loading in all trackways pickle')
# start_time = time.time()
# df = pd.read_pickle(vid_locations + 'AllTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)

# make list of trials
# %run LoadInTrackedData.ipynb # loads all functions
# from ipynb.fs.defs.LoadInTrackedData import create_trial_info # loads only one function
# trial_info = create_trial_info(file_list)

# load in dataframe of long trackways
# start_time = time.time()
# longtracks = pd.read_pickle(vid_locations + 'LongTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)

loading in all trackways pickle
   colony: 90813
   colony: 90814
   colony: 90815
   colony: 90816
   colony: 90817
   colony: 90821
   colony: 90822
   colony: 90823
   colony: 90824
read in all trackways pickle -- duration:  16.577251434326172

Removed trackways with a weirdly high velocity value


# OPTIONAL: BUILD DATA SET AND SAVE AS PICKLE


In [ ]:
# FOR EACH COLONY, BUILD DATA AND SAVE AS PICKLE

colonies = glob.glob(os.path.join(vid_locations, 'Tunnel_2019**/'))
# %run LoadInTrackedData.ipynb # load in function of interest
from ipynb.fs.defs.LoadInTrackedData import load_and_analyze_videos_to_df
buffer = 0 #pix - remove sections when ant this this close to frame edge

for col in colonies:
    
    f_list = sorted(glob.glob(os.path.join(col, '**/*0000.mp4')))
    print('\n%s - %i videos'%(col,len(f_list)))
    if len(f_list)>0:
        load_and_analyze_videos_to_df(f_list, buffer)
    

# # LOAD IN DATA
# %run LoadInTrackedData.ipynb
# df, trial_info = load_and_analyze_videos_to_df(file_list)
# # df, trial_info = load_and_analyze_videos_to_df(file_list)


# # if just building trial list
# # trial_info = create_trial_info(file_list)

del f_list, col, colonies

print('ALL DONE!!! YAY!')

#### Find long trials and bootstrap

In [ ]:
# FIND LONG TRIALS 
# import random


temp = df.copy()
# look only at trials longer than 50fr
idcs = [index for index, row in temp.iterrows() if len(row.v)>50]
longtracks = df.loc[idcs,]

# # MEDIAN V
# colony_R = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
# date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
# day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
# subs_string = longtracks['substrate'].values.tolist()
# substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
# v_med_R = np.array(longtracks['median_v'])
# df_med_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
#                        "v_med" : v_med_R } )

# del idcs, colony_R, day_R, substrate_R, v_med_R, date_days, subs_string, temp



# # BOOTSTRAP
# n_boot = 50;

# # make a numpy array of each random sample for one entry in dataframe
# sample = []
# for boot in range(0,100):
# #     sample.append( np.array([random.choice(tr) for tr in list(temp.v)]) )
#     sample.append( np.array([random.choice(row.v) if len(row.v)>50 else np.nan for index, row in longtracks.iterrows() ]))
    
# # convert sample list into np array
# allsamples = np.transpose(np.vstack(sample))
# samplesubset = allsamples[:,0:n_boot]

# # make new dataframe to be output to R as feather
# shortcolnames = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
# colony_R = np.repeat(np.array(shortcolnames), n_boot)
# date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
# col_days = [col.split('-').index(day) for day, col in zip(date_days, shortcolnames)]
# day_R = np.repeat(np.array(col_days), n_boot)
# subs_string = longtracks['substrate'].values.tolist()
# subs_int = np.array([int(s.split('mm')[0]) for s in subs_string])
# substrate_R = np.repeat(subs_int, n_boot)
# indiv_R = np.repeat(np.array(longtracks.index), n_boot)
# df_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
#                        "indiv" : indiv_R, "v" : np.concatenate(samplesubset)})

# del allsamples, samplesubset, shortcolnames, colony_R, day_R, substrate_R, date_days, subs_string, subs_int, indiv_R, sample
del temp
print('done')

#### Manually save things as pickle or feather

In [ ]:
# SAVE DATAFRAME AS FEATHER
# import time
# import feather

# # save master dataframe as pickle
# df_preference = (df>0).astype(np.int8)
# start_time = time.time()
# df.to_pickle(vid_locations + 'AllTracks')#, compression='infer', protocol=4) 
# print('Saved all trackways dataframe as pickle - duration: ', time.time()-start_time)

# save master dataframe with TDs for each colony
# colonies = glob.glob(os.path.join(vid_locations, 'Tunnel**/'))
# for col in colonies:
#     col_OI = col.split('/')[-2][-5:]
#     df.loc[(df['colony']!='Tunnel_201803'+col_OI)].to_pickle(vid_locations + '/AllTracks_TDs_%s'%col_OI, protocol = 4)#, compression='infer', protocol=4) 
#     print(col_OI)
# print('    Done saving pickle')
    
# save master dataframe as feather
# start_time = time.time()
# # newdf = df[['ID','colony','datetime','date','median_v','substrate','x_raw']].copy()
# # feather.write_dataframe(newdf, vid_locations + 'AllTracks.feather')
# print('Saved all trackways dataframe as feather - duration: ', time.time()-start_time)

# save long tracks dataframe as pickle
# start_time = time.time()
# longtracks.to_pickle(vid_locations + 'LongTracks')
# print('Saved long trackways dataframe as pickle - duration: ', time.time()-start_time)

# save long tracks dataframe as feather
# start_time = time.time()
# # feather.write_dataframe(longtracks, vid_locations + 'LongTracks.feather')
# print('Saved long trackways dataframe as feathers - duration: ', time.time()-start_time)

# print('Saved median and bootstrap files as feathers')
# feather.write_dataframe(df_med_R, vid_locations + 'Median.feather')
# # feather.write_dataframe(df_R, vid_locations + 'Bootstrap.feather')
# del df_med_R, df_R

# Functions that are needed in full file

In [5]:
def lowpass_filt_sections(arr, fr):
    yy = np.isnan(arr)
    xx = range(len(yy))
    full_filtered = np.empty(yy.shape)*np.nan
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == False: # if is a group of nan
            g = list(g)
#             print('section to lowpass fitler: ', len(g))
            if len(g)>9:
                b, a = signal.butter(2,fr,btype='low')
                new_arr = arr[np.array(g)]
                mirror_arr = np.hstack([np.flip(new_arr,axis=0), new_arr, np.flip(new_arr,axis=0)])
                filtered = signal.filtfilt(b, a, mirror_arr)
                full_filtered[np.array(g)]=filtered[len(new_arr):-1*len(new_arr)]
    return full_filtered

# Find when lights turned on and off for each recording session, add dataframe row

In [6]:
if os.path.exists(vid_locations + '/LightingInfo.pkl'):
    lighting_info = pd.read_pickle(vid_locations + '/LightingInfo.pkl')
    print('loaded lighting info from pickle')
    
else:
    print('calculating lighting info')
    rsessions = glob.glob(os.path.join(vid_locations, 'Tunnel_2019**/WebcamPhotos/'))
    temp = []
    for rsess in rsessions:
        tmp = {}
        print(rsess)
        tmp['date']=rsess.split('/')[-3].split('_')[1]


        pic_names = sorted(glob.glob(os.path.join(rsess, '**jpg')))
        is_light = np.ones([len(pic_names),2])
        for pp,pic in enumerate(pic_names):
            img = cv2.imread(pic)
            is_light[pp,1]=np.mean(img)>120
            is_light[pp,0]=int(pic.split('/')[-1].split('_')[1])
        is_light = is_light.astype(np.int)


        dark_to_light = np.diff(np.insert(is_light[:,1],0,0))==1
        light_to_dark = np.diff(np.insert(is_light[:,1],-1,0))==-1
        tmp['when_light'] = np.ones([np.sum(dark_to_light),2])
        tmp['when_light'][:,0]=is_light[dark_to_light,0]
        tmp['when_light'][:,1]=is_light[light_to_dark,0]

        light_to_dark2 = np.diff(np.insert(is_light[:,1],0,1))==-1
        dark_to_light2 = np.diff(np.insert(is_light[:,1],-1,1))==1
        tmp['when_dark'] = np.ones([np.sum(dark_to_light2),2])
        tmp['when_dark'][:,1]=is_light[dark_to_light2,0]
        tmp['when_dark'][:,0]=is_light[light_to_dark2,0]

        temp.append(tmp)
        del tmp, dark_to_light, dark_to_light2, light_to_dark, light_to_dark2, is_light, pic_names
    lighting_info = pd.DataFrame(temp)
    del temp
    print('done calculating lighting info')

    print('\nsaving lighting info as pickle')
    lighting_info.to_pickle(vid_locations + '/LightingInfo.pkl', protocol = 4)

loaded lighting info from pickle


In [ ]:
# assign each trial from dataframe to light or dark


def find_light_dark_df(x, coltypes, max_s_from_webcam_photo):
    time = int(x['time'])
    col_OI = x['colony']
    col_idx = np.where(np.isin(coltypes,col_OI))[0][0]
    light = lighting_info['when_light'][col_idx]
    dark = lighting_info['when_dark'][col_idx]
    is_light = np.any(np.logical_and(time>=light[:,0], time <= light[:,1]))
    is_dark = np.any(np.logical_and(time>=dark[:,0], time <= dark[:,1]))
    assignment = np.nan
    
    if is_light | is_dark:
        if is_light & is_dark:
            print('Col: %s, Time: %i -- unassigned -- lies within both ranges'%(col_OI, time))
        else:
            assignment = np.array([1,0])[np.array([is_light, is_dark])][0]
            
    else:
        how_close_to_light = np.min(np.abs(time-light))
        how_close_to_dark = np.min(np.abs(time-dark))
        if how_close_to_light < how_close_to_dark:
            if how_close_to_light <=5:
                print('Col: %s, Time: %i -- Assigned to light -- light/dark: %i/%i'%(col_OI, time, how_close_to_light, how_close_to_dark))
                assignment = 1
        elif how_close_to_light > how_close_to_dark:
            if how_close_to_dark <=5:
                print('Col: %s, Time: %i -- Assigned to dark -- light/dark: %i/%i'%(col_OI, time, how_close_to_light, how_close_to_dark))
                assignment = 0
        else:
            print('Col: %s, Time: %i -- unassigned -- light/dark: %i/%i'%(col_OI, time, how_close_to_light, how_close_to_dark))
            
    return assignment
    
    
columns_to_drop = ['light']
for colmn in columns_to_drop:
    if colmn in df: # remove columns if already exist
        df = df.drop(colmn, axis = 1)

coltypes = sorted(list(set(df['colony'])))
df = df.reindex( columns = df.columns.tolist() + ['light'] )
df['light'] = df.apply(
    find_light_dark_df, args = (coltypes,5), axis=1)
    

In [439]:
# try out assignment method for specific trials
for tr in np.arange(0,10):
    time = int(df['time'][tr])
    col_OI = df['colony'][tr]
    col_idx = np.where(np.isin(coltypes,col_OI))[0][0]
    light = lighting_info['when_light'][col_idx]
    dark = lighting_info['when_dark'][col_idx]
    is_light = np.any(np.logical_and(time>=light[:,0], time <= light[:,1]))
    is_dark = np.any(np.logical_and(time>=dark[:,0], time <= dark[:,1]))

    if is_light & is_dark:
        print('ERROR: lies within both ranges!')
    if (not is_light) & (not is_dark):
        how_close_to_light = np.min(np.abs(time-light))
        how_close_to_dark = np.min(np.abs(time-dark))
        
        if how_close_to_light < how_close_to_dark:
            if how_close_to_light <=5:
                print('Tr: %i, Time: %i -- Assigned to light -- light/dark: %i/%i'%(tr, time, how_close_to_light, how_close_to_dark))
        elif how_close_to_light > how_close_to_dark:
            if how_close_to_dark <=5:
                print('Tr: %i, Time: %i -- Assigned to dark -- light/dark: %i/%i'%(tr, time, how_close_to_light, how_close_to_dark))
        else:
            print('Tr: %i, Time: %i -- unassigned -- light/dark: %i/%i'%(tr, time, how_close_to_light, how_close_to_dark))
            
        
#         if (how_close_to_light<=5)
# #         is_light = np.any(np.logical_and(time>=(light[:,0]-2), time<=(light[:,1]+2)))
# #         is_dark = np.any(np.logical_and(time>=(dark[:,0]-2), time<=(dark[:,1]+2)))
# #     is_dark = np.any(np.logical_and(time>dark[:,0], time < dark[:,1]))
#         print('ERROR: lies in neither range')
#         print('Tr: %i, Time: %i -- light/dark: %i/%i'%(tr, time, how_close_to_light, how_close_to_dark))

# Get rid of points where ant is close to a horizontal edge of the substrate 

In [47]:
# load in information from pickles saved from Find_Array_Edges notebook
vid_locations = '/media/gravishlab/SeagateExpansionDrive/Light_Dark/'
sub_edge_files = glob.glob(os.path.join(vid_locations, '**/*[!s]/*_Edges.pkl'))
sub_edge_files = sorted(sub_edge_files)
print('Total Number of Videos: ',len(sub_edge_files))

sub_edge_data = []
for file in sub_edge_files:
    colony = file.split('/')[-3]
    substrate = file.split('/')[-2]
#     print('%s -- %s'%(colony, substrate))
    temp = {}
    with open(file, 'rb') as f:
        temp['vlines'], temp['hlines'], temp['top_step_loc'], temp['img_raw'] = pickle.load(f)
    f.close()
    temp['colony']=colony
    temp['substrate']=substrate
    sub_edge_data.append(temp)
sub_edge_df = pd.DataFrame(sub_edge_data).copy()

del temp, sub_edge_data, sub_edge_files, colony, substrate

print('loaded in edge data for all substrates and colonies')

Total Number of Videos:  36
loaded in edge data for all substrates and colonies


In [48]:
# apply to whole dataset

def remove_close_to_edge_df(df, buffer_mm, pix2mm, sub_edge_df, wid):
    buffer = buffer_mm*pix2mm
    col = df['colony']
    sub = df['substrate']
    idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
    p = sub_edge_df.iloc[idx]['hlines'].values[0]

    x = np.array(df['x_raw'])
    y = np.array(df['y_raw'])
    v = np.append(np.array(df['vfilt']), np.nan)
    
    # get rid of crazy accelerations from bad tracking
    tmp = np.diff(v/pix2mm); tmp[-1] = tmp[-2]; tmp = tmp>15
    idcs = np.logical_or(np.insert(tmp, 0, False), np.append(tmp, False))
    x[idcs] = np.nan; y[idcs] = np.nan; v[idcs]=np.nan
    
    near_bottom = y > (p[0] + p[1]*x - buffer)
    near_top = np.logical_or(y < (p[0] + p[1]*x - 16*pix2mm + buffer), y < buffer)
    near_left = x < 3*pix2mm
    near_right = x > wid-3*pix2mm
    remove = np.logical_or(np.logical_or(near_bottom, near_top),  np.logical_or(near_left, near_right))
    x[remove] = np.nan; y[remove] = np.nan; v[remove] = np.nan
    return x,y,v

def take_moving_ave(x, N, n_nan_cutoff):
    padded_x = np.insert(np.insert( np.insert(x, len(x), np.empty(int(N/2))*np.nan), 0, np.empty(int(N/2))*np.nan ),0,0)
    n_nan = np.cumsum(np.isnan(padded_x))
    cumsum = np.nancumsum(padded_x) 
    window_sum = cumsum[N+1:] - cumsum[:-(N+1)] #- x # subtract value of interest from sum of all values within window
    window_n_nan = n_nan[N+1:] - n_nan[:-(N+1)] #- np.isnan(x)
    window_n_values = (N - window_n_nan)
    movavg = (window_sum) / (window_n_values)
    movavg[window_n_nan>n_nan_cutoff]=np.nan
    return movavg

def remove_slow_df(df, win_wid, n_nan_cutoff, v_cutoff, pix2mm):

    x = np.array(df['x_final'])
    y = np.array(df['y_final'])
    v = np.array(df['v_final'])
    
#     movavg = take_moving_ave(v, win_wid, n_nan_cutoff)
#     v_remove = movavg/pix2mm<v_cutoff
    lowpass = np.append(lowpass_filt_sections(df['vfilt'], 0.06),np.nan)
    v_remove = lowpass/pix2mm<v_cutoff
    
    x[v_remove] = np.nan; y[v_remove] = np.nan; v[v_remove] = np.nan
#     v_movavg = movavg.copy()
#     v_movavg[v_remove] = np.nan
    v_lowpass = lowpass.copy()
    v_lowpass[v_remove] = np.nan
    return x,y,v, lowpass, v_lowpass


df['x_final'], df['y_final'], df['v_final'] = zip(*df.apply(
        remove_close_to_edge_df, args = (2, pix2mm, sub_edge_df, wid), axis=1))
print('done removing when ant close to edge of substrate or frame')

df['x_final_noslow'], df['y_final_noslow'], df['v_final_noslow'], df['v_final_lowpass'], df['v_final_lowpass_noslow'] = zip(*df.apply(
        remove_slow_df, args = (20, 4, 3, pix2mm), axis=1))
print('done removing when ant is stopped')

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater


done removing when ant close to edge of substrate or frame


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in less


done removing when ant is stopped


In [ ]:
# CHECK THAT GETTING GOOD FINAL VALUES FOR SPECIFIC TRIAL
tr = 1078

plt.close('all')
plt.figure()
plt.subplot(2,1,1)
plt.plot(df['x_raw'][tr],df['y_raw'][tr],'-k')
plt.plot(df['x_final'][tr],df['y_final'][tr],'.b')
plt.xlim([0,1000])
plt.ylim([0,550])

plt.subplot(2,1,2)
plt.plot(df['frames'][tr][:-1],df['vfilt'][tr]/pix2mm,'-k')
plt.plot(df['frames'][tr][:-1],df['vfilt'][tr]/pix2mm,'.k')
plt.plot(df['frames'][tr],df['v_final'][tr]/pix2mm,'-b')
plt.plot(df['frames'][tr],df['v_final_movavg_noslow'][tr]/pix2mm,'-r')
plt.ylim([0,100])


#TRIALS WITH CRAZY ACCELERATIONS: [673, 872, 1078, 1346, 1364, 3669, 4248, 4541, 4764]

In [169]:
# FOR SPECIFIC TRIAL - remove when ant close to substrate edge or slow
# df['x_kal'] <-- straight from kalman filter, no post-processing
# df['x_raw'] <-- lowpass filtered
# df['x'] <-- removed when ant slow or near wall

buffer = 1*pix2mm
# tr = 3552
for tr in np.arange(233,234):


    col = df['colony'][tr]
    sub = df['substrate'][tr]
    print('%s, %s'%(col, sub))
    idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
    img_raw = sub_edge_df.iloc[idx]['img_raw'].values[0]
    p = sub_edge_df.iloc[idx]['hlines'].values[0]

    x = np.array(df['x_raw'][tr])
    y = np.array(df['y_raw'][tr])
    near_bottom = y > (p[0] + p[1]*x - buffer)
    near_top = np.logical_or(y < (p[0] + p[1]*x - 16*pix2mm + buffer), y < buffer)
    near_left = x < 3*pix2mm
    near_right = x > wid-3*pix2mm
    remove = np.logical_or(np.logical_or(near_bottom, near_top),  np.logical_or(near_left, near_right))





    # remove when ant is slow
    win_wid = 12
    nfr_cutoff = 20
    v_cutoff = 3 # in mm/s
    movavg = take_moving_ave(df['vfilt'][tr], win_wid, nfr_cutoff)
    movavg = np.append(movavg, np.nan)
#     v_remove = movavg/pix2mm<v_cutoff
    lowpass = np.append(lowpass_filt_sections(df['vfilt'][tr], 0.06),np.nan)
    v_remove = lowpass/pix2mm<v_cutoff



    plt.close('all')
    plt.figure(figsize=(6,7))
    ax1=plt.subplot(2,1,1)
    plt.imshow(img_raw, cmap = 'gray')
    xd = np.arange(0,wid)
    sub_edge_df.iloc[idx]['img_raw'].values
    yd = p[0]+p[1]*xd
    plt.plot(xd,yd,'-g')
    plt.plot(xd,yd-buffer,':g')
    plt.plot(xd,yd-16*pix2mm+buffer,':g')
    plt.plot(x,y,'-k')
    plt.plot(x[remove],y[remove],'.b')


    plt.subplot(2,1,2)
    plt.plot(df['frames'][tr][:-1], df['vfilt'][tr]/pix2mm, '-k')
    plt.plot(df['frames'][tr], movavg/pix2mm, '-r')
    plt.plot(df['frames'][tr], lowpass/pix2mm, '-g')
    plt.axhline(y=v_cutoff, xmin=0, xmax=1, color ='r', alpha = 0.3)
    plt.axhline(y=5, xmin=0, xmax=1, color ='g', alpha = 0.3)
    plt.sca(ax1)
    plt.plot(x[v_remove],y[v_remove],'.r')

    plt.pause(3)


Tunnel_20190813, Step


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in less


## Improve Angle approximation from tracking, calculate forward and lateral velocity

In [52]:
# FILP ANGLE FUNCTIONS

def calc_mov_avg(x, N, cutoff_nan):
    padded_x = np.insert(np.insert( np.insert(x, len(x), np.empty(int(N/2))*np.nan), 0, np.empty(int(N/2))*np.nan ),0,0)
    n_nan = np.cumsum(np.isnan(padded_x)) 
    cumsum = np.nancumsum(padded_x) 
    window_sum = cumsum[N+1:] - cumsum[:-(N+1)] - x
    window_n_nan = n_nan[N+1:] - n_nan[:-(N+1)] - np.isnan(x)
    window_n_values = (N - window_n_nan).astype(float)
    window_n_values[window_n_values<cutoff_nan] = np.nan # if fewer than cutoff values in window, ignore
    movavg = (window_sum) / (window_n_values)
    return movavg

def remove_90_turns(arr, all_frs):
        idcs = np.where(np.isfinite(arr))[0]
        d_arr = np.diff(arr[idcs])
        d_n_fr = np.diff(idcs)
        d_sign = np.sign(d_arr)
        d_med = np.logical_and(np.logical_and( np.abs(d_arr) > 0.5, np.abs(d_arr) < 1.5), d_n_fr<10)
        d_med_cumsum = np.cumsum(np.insert(d_med,0,0))
        d_med_opp = np.cumsum(np.insert(d_sign*d_med,0,0))#(d_med_cumsum%2).astype(bool)
        if np.sum(d_med_opp==0)<np.sum(d_med_opp != 0):
            d_med_opp = d_med_opp - np.median(d_med_opp)
        arr[idcs]=np.clip(arr[idcs]-d_med_opp, -1, 1)  
        return arr

def find_flipped_angles_all(obj_OI, plots_on = False):
    
    all_angs = np.array(obj_OI['angle_improved'])
    all_angs[np.isnan(all_angs)]= np.array(obj_OI['angle'])[np.isnan(all_angs)]-90
#     all_angs = obj_OI['angle']-90
    
    
    all_frs = obj_OI['frames']
    all_sin = np.sin(np.deg2rad(all_angs))
    all_cos = np.cos(np.deg2rad(all_angs))
    all_sin_sign = np.sign(np.nanmedian(all_sin))
    all_cos_sign = np.sign(np.nanmedian(all_cos))
    
    
    if np.sum(np.isfinite(all_angs))==0:
        return np.ones(np.array(all_angs).shape)*np.nan
    
    if plots_on:
        plt.figure()
        plt.subplot(3,1,1)
        plt.plot(all_frs, all_sin, '-k', alpha = 0.3)
        plt.plot(all_frs, all_sin, '.g', alpha = 0.3)
        plt.plot(all_frs, all_cos, '-k', alpha = 0.3)
        plt.plot(all_frs, all_cos, '.b', alpha = 0.3)

    # FLIP EVERY OTHER SECTION DEFINED WHERE COS OR SIN CHANGES BY MORE THAN 1
    idcs = np.where(np.isfinite(all_sin))[0]
    d_sin = np.abs(np.diff(all_sin[idcs]))
    d_cos = np.abs(np.diff(all_cos[idcs]))
    d_big = np.logical_or(d_sin> 1.5, d_cos > 1.5)
    d_big_cumsum = np.cumsum(np.insert(d_big,0,0))
    d_big_opp = (d_big_cumsum%2).astype(bool)
    
    if plots_on:
        plt.plot(all_frs[idcs[d_big_opp]], all_sin[idcs[d_big_opp]], '.r', MarkerSize = 2)
        plt.plot(all_frs[idcs[d_big_opp]], all_cos[idcs[d_big_opp]], '.r', MarkerSize = 2)
        plt.ylim([-1.1,1.1])
    
    all_sin[idcs[d_big_opp]] = -1* all_sin[idcs[d_big_opp]]
    all_cos[idcs[d_big_opp]] = -1* all_cos[idcs[d_big_opp]]
    
#     # if is now wrong sign, flip whole array
#     if (np.sign(np.nanmedian(all_sin)) != all_sin_sign) & (np.sign(np.nanmedian(all_cos)) != all_cos_sign):
# #         print('switching', all_sin_sign, np.sign(np.nanmedian(all_sin)))
#         all_sin = -1* all_sin
#         all_cos = -1* all_cos
        
    all_sin = remove_90_turns(all_sin, all_frs)
    all_cos = remove_90_turns(all_cos, all_frs)
    
    
    # interpolate nan gaps
    nan_gap_fill = 3
    where_interp =find_nan_gaps(np.array(all_angs), nan_gap_fill)
    if np.sum(where_interp)>0:
        all_sin = interp_vals(all_sin, find_interp_idcs(where_interp))
        all_cos = interp_vals(all_cos, find_interp_idcs(where_interp))
    
        
    all_sin_good = all_sin.copy()
    all_cos_good = all_cos.copy()
#     all_sin_smooth = calc_mov_avg(all_sin_good, 6, 2)
#     all_cos_smooth = calc_mov_avg(all_cos_good, 6, 2)
    fr_cutoff = 0.05
    all_sin_smooth = lowpass_filt_sections(all_sin_good, fr_cutoff)
    all_cos_smooth = lowpass_filt_sections(all_cos_good, fr_cutoff)
    all_angs_good = np.rad2deg(np.arctan2(all_sin_smooth, all_cos_smooth))

    if plots_on:
        plt.subplot(3,1,2)
        plt.plot(all_frs, all_sin_good, '-k', alpha = 0.3)
        plt.plot(all_frs, all_sin_good, '.k', alpha = 0.3)
        plt.plot(all_frs, all_cos_good, '-k', alpha = 0.3)
        plt.plot(all_frs, all_cos_good, '.k', alpha = 0.3)
        plt.plot(all_frs, all_sin_smooth, '.g', alpha = 0.3)
        plt.plot(all_frs, all_cos_smooth, '.b', alpha = 0.3)
        plt.ylim([-1.1,1.1])
        plt.subplot(3,1,3)
        plt.plot(all_frs, all_angs, 'k', alpha = 0.3)
        plt.plot(all_frs, all_angs_good, 'r')
        plt.ylim([-180, 180])
    return all_angs_good


# average x,y,angle if isolated frame without head found (angle_improve = nan)
def find_nan_gaps(arr, limit):  
    from itertools import groupby
    yy = np.isnan(arr)
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
                continue       
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
    return where_gapOI
def find_interp_idcs(where_interpolate):
    interp_idcs = []
    for val in [-1,0,1]:
        interp_idcs = np.concatenate([interp_idcs,np.where(where_interpolate)[0]+val])
    interp_idcs = np.sort(np.array(list(set(interp_idcs)))) # get of repeat elements
    interp_idcs = interp_idcs[np.logical_and(interp_idcs>-1, interp_idcs < len(where_interpolate))].astype(np.uint32) # only elements in range
    return interp_idcs
def interp_vals(arr, interp_idcs): # array includes nan values
    temp = arr[interp_idcs]
    interpolated_vals = np.interp(
        interp_idcs, 
        interp_idcs[np.logical_not(np.isnan(temp))], temp[np.logical_not(np.isnan(temp))] )
    interp = arr.copy()
    interp[interp_idcs] = interpolated_vals
    return interp


In [53]:
# apply to whole dataframe

def fix_angles_df(obj_OI, plot_things, nan_gap_fill):
    tmp = find_flipped_angles_all(obj_OI, plots_on = plot_things)
    where_interp =find_nan_gaps(tmp, nan_gap_fill)
    if np.sum(where_interp)>0:
        tmp = interp_vals(tmp, find_interp_idcs(where_interp))
    return tmp

df['angle_smooth'] =  df.apply(fix_angles_df, args =(False, 4), axis=1)
print('done fixing and smoothing orientation angles in dataframe')

# # one trial
# _=fix_angles_df(df.loc[4652], True, 4)

done fixing and smoothing orientation angles in dataframe


In [54]:
def calc_forward_vel_df(df, v_multiplier, var_string):
    frames = df['frames']
    t_x = df['x_%s'%var_string]
    t_y = df['y_%s'%var_string]
    d_t_x, d_t_y = np.diff(t_x), np.diff(t_y)
    travel_dist = np.linalg.norm([d_t_x, d_t_y], axis =0)

    or_start = (np.deg2rad(df['angle_smooth'])[:-1]+2*np.pi)%(2*np.pi)%(2*np.pi)
    or_stop = (np.deg2rad(df['angle_smooth'])[1:]+2*np.pi)%(2*np.pi)%(2*np.pi)
    or_ave = (np.arctan2(np.nanmean(np.array([np.sin(or_start), np.sin(or_stop)]), axis=0), 
                            np.nanmean(np.array([np.cos(or_start), np.cos(or_stop)]),axis = 0))+2*np.pi)%(2*np.pi)%(2*np.pi)
    tr_dir = (np.arctan2(d_t_y, d_t_x)+(2*np.pi))%(2*np.pi)
    
    v_forward = np.linalg.norm([d_t_x, d_t_y], axis =0)*np.cos(tr_dir - or_ave)*v_multiplier
    v_lateral = np.sin(tr_dir - or_ave)*travel_dist*v_multiplier
    return v_forward, v_lateral

# calculate forward and lateral from final x and y
df['v_forward'], df['v_lateral'] = zip(*df.apply(
        calc_forward_vel_df, args = (v_multiplier,'final'), axis=1))
print('calculated first round of forward and lateral velocities')

# some angles might be totally switched (pointing to back of ant) -- find and fix
df['test']=df.apply(lambda x: x.angle_smooth + 180 if np.nanmedian(x.v_forward) <= -1 else x.angle_smooth, axis =1)
df.drop("angle_smooth", axis=1, inplace=True)
df.rename(columns = {'test':'angle_smooth'}, inplace=True)
print('fixed any trials where ant is mostly moving backwards')

# recalculate forward and lateral velocities
df['v_forward'], df['v_lateral'] = zip(*df.apply(
        calc_forward_vel_df, args = (v_multiplier,'final'), axis=1))
print('recalculated forward and lateral velocities')

# calculate forward and lateral without slow points
df['v_forward_noslow'], df['v_lateral_noslow'] = zip(*df.apply(
        calc_forward_vel_df, args = (v_multiplier,'final_noslow'), axis=1))
print('recalculated forward and lateral velocities without slow points')

# lowpass filter forward and lateral velocity without slow points
df['v_forward_lowpass_noslow'] = df['v_forward_noslow'].apply(lowpass_filt_sections, args = (0.06,) )
df['v_lateral_lowpass_noslow'] = df['v_lateral_noslow'].apply(lowpass_filt_sections, args = (0.06,) )
print('lowpass filtered forward and lateral velocities without slow points')


print('\n\nDone!')

calculated first round of forward and lateral velocities


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


fixed any trials where ant is mostly moving backwards
recalculated forward and lateral velocities
recalculated forward and lateral velocities without slow points
lowpass filtered forward and lateral velocities without slow points


Done!


In [181]:
tr=0
plt.close('all')
plt.figure()
plt.plot(np.deg2rad(df['angle_smooth'][tr]), '-k')


In [255]:
tr = 4652
var_string = 'final_noslow'
frames = df['frames'][tr]
t_x = df['x_%s'%var_string][tr]
t_y = df['y_%s'%var_string][tr]
d_t_x, d_t_y = np.diff(t_x), np.diff(t_y)
travel_dist = np.linalg.norm([d_t_x, d_t_y], axis =0)

or_start = (np.deg2rad(df['angle_smooth'][tr])[:-1]+2*np.pi)%(2*np.pi)%(2*np.pi)
or_stop = (np.deg2rad(df['angle_smooth'][tr])[1:]+2*np.pi)%(2*np.pi)%(2*np.pi)
or_ave = (np.arctan2(np.nanmean(np.array([np.sin(or_start), np.sin(or_stop)]), axis=0), 
                        np.nanmean(np.array([np.cos(or_start), np.cos(or_stop)]),axis = 0))+2*np.pi)%(2*np.pi)
tr_dir = (np.arctan2(d_t_y, d_t_x)+(2*np.pi))%(2*np.pi)

plt.figure()
plt.plot(or_start)
plt.plot(tr_dir)

v_forward = np.linalg.norm([d_t_x, d_t_y], axis =0)*np.cos(tr_dir - or_ave)*v_multiplier
v_lateral = np.sin(tr_dir - or_ave)*travel_dist*v_multiplier

# Find Step direction and travel direction

In [61]:
# find step direction
step_df = df.loc[df['substrate'].isin(['Step','C_Step'])]

def find_step_direction(df, sub_edge_df):
    step_dir = np.nan; c_idx = np.nan; c_x = np.nan; c_y = np.nan; t_dir = np.nan
    
    sub= df['substrate']
    col = df['colony']
    idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
    img_raw = sub_edge_df.iloc[idx]['img_raw'].values[0]
    step_side = sub_edge_df.iloc[idx]['top_step_loc'].values[0]
    step_side_val = np.array([1,-1])[np.isin(['right','left'],step_side)]
    p = sub_edge_df.iloc[idx]['vlines'].values[0][1]

    # get pathway info
    x = df['x_final']; y = df['y_final']; v = df['v_final'];

    # does get within __ mm of center?
    center_cutoff = 0.5
    dist_from_center_cutoff = 5
    cross_center = np.any(np.abs(x-(y-p[0])/p[1])<center_cutoff*pix2mm)
    travel_far = np.any(np.abs(x-(y-p[0])/p[1])>dist_from_center_cutoff*pix2mm)


    if cross_center and travel_far:
        # overall direction moving
        tmp = np.diff(x)
        bin_counts,_ = np.histogram(tmp[np.isfinite(tmp)],bins = [-5,-1,1,5])
        if np.abs( (bin_counts[2]+1)/(bin_counts[0]+1))<10: # if not a clear travel direction don't include in analysis
            return t_dir, step_dir, c_idx, c_x, c_y
        bin_counts[1]=0
        t_dir = np.argmax(bin_counts)-1 #np.sign(np.nanmedian(np.diff(x))) # -1 = to left, 1 = to right
        step_dir = t_dir*step_side_val # -1 = down, 1 = up
        step_dir = step_dir[0]
        
        # when cross center
        dist_from_step = x-(y-p[0])/p[1]
        c_idcs = np.argsort(np.abs(dist_from_step))[0:2]
        c_idx = c_idcs[0]
        c_x = x[c_idcs][0]-dist_from_step[c_idcs][0]
        c_y = p[0]+c_x*p[1]
    return t_dir, step_dir, c_idx, c_x, c_y
    
    
step_df['travel_dir'], step_df['step_dir'], step_df['cross_step_idx'], step_df['cross_step_x'], step_df['cross_step_y'] = zip(*step_df.apply(
    find_step_direction, args = (sub_edge_df,), axis=1))

print('done saving step direction and crossing info')

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done saving step direction and crossing info


In [56]:
# for one trial --> find if step trial crosses center

step_df = df.loc[df['substrate'].isin(['Step','C_Step'])]

tr = 2960

sub= step_df['substrate'].loc[tr]
col = step_df['colony'].loc[tr]
print('%s -- %s'%(col, sub))

# get substrate edge info
idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
img_raw = sub_edge_df.loc[idx]['img_raw'].values[0]
step_side = sub_edge_df.loc[idx]['top_step_loc'].values[0]
step_side_val = np.array([1,-1])[np.isin(['right','left'],step_side)]
p = sub_edge_df.loc[idx]['vlines'].values[0][1]

# get pathway info
x = step_df['x_final'].loc[tr]; y = step_df['y_final'].loc[tr]; v = step_df['v_final'].loc[tr]; 
v_movavg = step_df['v_final_lowpass'].loc[tr]; frames = step_df['frames'].loc[tr];

# does get within __ mm of center?
center_cutoff = 0.5
dist_from_center_cutoff = 6
cross_center = np.any(np.abs(x-(y-p[0])/p[1])<center_cutoff*pix2mm)
travel_far = np.any(np.abs(x-(y-p[0])/p[1])>dist_from_center_cutoff*pix2mm)
print('crosses center? %i -- travels at least %i mm? %i'%(cross_center, dist_from_center_cutoff, travel_far))



if cross_center and travel_far:
    
    # overall direction moving
    t_dir = np.sign(np.nanmedian(np.diff(x))) # -1 = to left, 1 = to right
    step_dir = t_dir*step_side_val # -1 = down, 1 = up
    print('stepping up(+) or down(-): %i'%step_dir)
    
    # when cross center
    dist_from_step = x-(y-p[0])/p[1]
    c_idcs = np.argsort(np.abs(dist_from_step))[0:2]
    c_idx = c_idcs[0]
    c_x = x[c_idcs][0]-dist_from_step[c_idcs][0]
    c_y = p[0]+c_x*p[1]
    
    

plt.close('all')
plt.figure(figsize=(6,7))
ax1=plt.subplot(2,1,1)
plt.imshow(img_raw, cmap = 'gray')


yd = np.arange(0,hei)
xd = (yd-p[0])/p[1]

plt.plot(xd,yd,'-g')
plt.plot(xd+center_cutoff*pix2mm,yd,':g')
plt.plot(xd-center_cutoff*pix2mm,yd,':g')
plt.plot(xd+dist_from_center_cutoff*pix2mm,yd,':g')
plt.plot(xd-dist_from_center_cutoff*pix2mm,yd,':g')

plt.plot(x,y,'-k')
plt.xlim([0,1000])
plt.ylim([550,0])


plt.subplot(2,1,2)
# plt.plot(step_df['frames'].iloc[tr],v,'-k')
plt.plot(x/pix2mm,v,'-k')
plt.plot(x/pix2mm,v_movavg,'--r')
plt.axvline(x=x[c_idx]/pix2mm,color='g')
plt.xlim([0,32])



Tunnel_20190821 -- Step
crosses center? 1 -- travels at least 6 mm? 1
stepping up(+) or down(-): 1


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


(0, 32)

# PLOT THINGS

## Plot distance traveled at each mov ave speed for light dark on flat and array

In [375]:
def get_dist_traveled_df(x, var_OI, bins):
    glenna=x[var_OI]#/pix2mm
    glenna= glenna[np.isfinite(glenna)]
    ordering = np.argsort(np.digitize(glenna,bins))
    ordered = glenna[ordering]
    split = np.split(ordered,np.where(np.diff(np.digitize(glenna,bins)[ordering])>0)[0]+1)
    split_sum = np.array([np.sum(x) for x in split])
    bin_idcs = np.unique(np.digitize(glenna,bins))
    bin_sums = np.zeros(bins.shape)
#     print(bin_idcs, [x[0] for x in split])
    bin_sums[bin_idcs.astype(int)] = split_sum/fps
    
    return np.abs(bin_sums)

longtracks = df[df['v_forward_lowpass_noslow'].map(lambda x: np.sum(np.isfinite(x))>50).values]

var_OI = 'v_forward_lowpass_noslow'#'movave_v'
sp_max = 60
precision = 10 # how many bins per 1 unit
bins = np.linspace(-20,sp_max, sp_max*precision+1)
longtracks['hist_dist'] =df.apply( get_dist_traveled_df, args = (var_OI, bins), axis=1)
subtypes = sorted(list(set(df['substrate'])))

plt.close('all')
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
percent_cutoff = 0.02
plt.figure(figsize = (12,6))

all_hists = np.ones([4,len(bins)])*np.nan


for ss,subtype in enumerate(['Flat', 'Array']):
    ax = plt.subplot(2,1,ss+1)
    print(' -- ', subtype)
#     for ltype in [1]:
#     sub_df = longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['light']==ltype)]
    sub_df = longtracks.loc[(longtracks['substrate']==subtype)]

    # plot dark trials
    vals_OI = sub_df.loc[(sub_df['light'].map(lambda x: x==0)).values]['hist_dist']
    total_dist_traveled = np.sum(vals_OI.sum())
    hist_OI = vals_OI.sum()/total_dist_traveled
    
    print(' -- -- total dist: %0.2f mm'%total_dist_traveled)
    kde_data = np.repeat(bins+1/(2*precision),np.round((hist_OI*10000)).astype(int))
    kde = stats.gaussian_kde(kde_data)
    kde_fit = kde.evaluate(bins)/precision
    all_hists[ss*2+0,:]=hist_OI*total_dist_traveled
#     if ss == 0:
#         ref_speed = bins[np.argmax(kde_fit)]+1/(2*precision)
#         plt.xlabel('speed (mm/s)')
#         plt.ylabel('fraction of dist traveled')
#     plt.axvline(x=ref_speed, ymin = 0, ymax = 1, color = 'k', linestyle = ':', alpha = 0.4)
#     top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
#     plt.axvline(x=top_percent, ymin = 0, ymax = 0.6, color = 'k', linestyle = '-', alpha = 0.4)
    plt.plot(bins[hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = 'k', alpha = 0.4)
    plt.text(40, 0.11/precision, '%i'%len(vals_OI), color = 'k', alpha = 0.4)
    plt.text(50, 0.11/precision, '%0.2f cm'%(total_dist_traveled/10), color = 'k', alpha = 0.4)
    plt.xlim([-10,sp_max])
    plt.ylim([0, 0.12/precision])

    # plot light trials
    vals_OI = sub_df.loc[(sub_df['light'].map(lambda x: x==1)).values]['hist_dist']
    total_dist_traveled = np.sum(vals_OI.sum())
    hist_OI = vals_OI.sum()/total_dist_traveled
    
    print(' -- -- total dist: %0.2f mm'%total_dist_traveled)
    kde_data = np.repeat(bins+1/(2*precision),np.round((hist_OI*10000)).astype(int))
    kde = stats.gaussian_kde(kde_data)
    kde_fit = kde.evaluate(bins)/precision
    all_hists[ss*2+1,:]=hist_OI*total_dist_traveled
    plt.plot(bins[hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '--', color = pltcolors[ss], alpha = 0.5)
#     top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
#     plt.axvline(x=top_percent, ymin = 0, ymax = 0.6, color = pltcolors[ss], linestyle = '--', alpha = 0.5)
    plt.text(40, 0.10/precision, '%i'%len(vals_OI), color = pltcolors[ss], alpha = 0.5)
    plt.text(50, 0.10/precision, '%0.2f'%(total_dist_traveled/10), color = pltcolors[ss], alpha = 0.5)
    
    plt.ylim([0, 0.12/precision])
    plt.ylabel('fraction of dist traveled')
    plt.xlabel('speed (mm/s)')
    plt.xlim([-10,sp_max])
    plt.text( 2, 0.11/precision, 'Substrate: %s'%subtype, color = 'k', size = 12)
    if ss == 0:
        plt.gca().get_xaxis().set_visible(False)


# plt.savefig('cinnamon_distance.svg', transparency = True)

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


 --  Flat
 -- -- total dist: 5369.51 mm
 -- -- total dist: 9385.61 mm
 --  Array
 -- -- total dist: 5483.80 mm
 -- -- total dist: 9070.94 mm


In [376]:
# Calculate if speed distributions are different in light and dark

from scipy.stats import mannwhitneyu
resolution_factor = 1 # how many mm should equal one datapoint
dists = np.array(['F-D', 'F-L', 'A-D', 'A-L'])

for pp,pairs in enumerate([[0,1],[2,3],[0,2],[1,3]]):
    tmp = np.repeat(bins, np.round(all_hists[pairs[0],:]/resolution_factor).astype(int))
    tmp2 = np.repeat(bins, np.round(all_hists[pairs[1],:]/resolution_factor).astype(int))
    F,p = mannwhitneyu(tmp, tmp2, alternative = 'two-sided')
    print('Compare %s with %s: F= %0.1f, p= %0.7e'%(dists[pairs[0]], dists[pairs[1]], F,p))


#### USE CONTINGENCY TABLES
# from scipy.stats import chi2_contingency
# # test if light diff on flat
# non_zero_cols = np.max(all_hists[:2,:]>0,axis=0)
# c2,p,dof,_ = chi2_contingency(all_hists[:2,non_zero_cols])
# print(c2,p)

# # test if light diff on array
# non_zero_cols = np.max(all_hists[2:,:]>0,axis=0)
# c2,p,dof,_ = chi2_contingency(all_hists[2:,non_zero_cols])
# print(c2,p)


# non_zero_cols = np.max(all_hists[1::2,:]>0,axis=0)
# c2,p,dof,_ = chi2_contingency(all_hists[1::2,non_zero_cols])
# print(c2,p)

# non_zero_cols = np.max(all_hists[0::2,:]>0,axis=0)
# c2,p,dof,_ = chi2_contingency(all_hists[0::2,non_zero_cols])
# print(c2,p)

Compare F-D with F-L: F= 22649828.0, p= 1.7423776e-21
Compare A-D with A-L: F= 23989869.0, p= 5.3295667e-03
Compare F-D with A-D: F= 25022545.5, p= 0.0000000e+00
Compare F-L with A-L: F= 73284039.5, p= 0.0000000e+00


## CALCULATE FRACTAL DIMENSIONS

In [265]:
# apply to full dataframe
def get_fractal_sinuosity_df(df, n_frs_cutoff, n_mm_total_travel, pix2mm, n_pix_cutoff): # how many pts does section need to be analyzed, how many pix to be identified frame
    x = np.array(df['x_final']) # 'x' has jumps from removing frames close to edge and when ant is stopped for long time
    y = np.array(df['y_final'])
    frames = df['frames']
    idcs = np.logical_and(np.isfinite(x), np.isfinite(y))
    x=x[idcs]; y = y[idcs]; frames= frames[idcs]

    # split into contiguous sections
    sections = np.split(np.arange(0,len(x)), np.where(np.diff(frames)>1)[0]+1)

    # make a richardson plot for each section to calc fractal dimension and sinuosity
    D = np.ones(len(sections))*np.nan
    P = np.ones(len(sections))*np.nan
    for ss,section in enumerate( sections):

        # dont' calculate things if fewer than 20 frames of data in section
        if len(section)<n_frs_cutoff:
#             print('section is too short')
            continue
            
        # total dist
        total_dist = np.cumsum(np.insert(np.linalg.norm([np.diff(x[section]),np.diff(y[section])], axis = 0),0,0))
        
        # don't calculate if total distance traveled in less than __ mm
        if total_dist[-1]<n_mm_total_travel*pix2mm:
#             print('ant travel distance is too short')
            continue
        
        # for each step distance, find total length
        dimensions = np.append(np.arange(5, total_dist[-1]/2, np.ceil(total_dist[-1]/15)), total_dist[-1]-0.01)
        total_lengths = np.ones(dimensions.shape)*np.nan
        for dd, dimen in enumerate(dimensions):
            errors = np.repeat(total_dist[:,np.newaxis], total_dist[-1]/dimen+1,axis=1
                              ) - np.repeat(np.arange(0,total_dist[-1], dimen)[np.newaxis,:], len(total_dist), axis =0)
            min_errors = np.min(np.abs(errors), axis =0)
            if np.any(min_errors > n_pix_cutoff):
#                 print('step length does not have data within cutoff for all steps')
                continue
            step_locations = np.argmin(np.abs(errors), axis =0)
            total_lengths[dd] = np.sum(np.linalg.norm([np.diff(x[section][step_locations]),np.diff(y[section][step_locations])], axis = 0))

        
        # find slope of all points not bookends --> calc fractal dist
        if np.sum(np.isfinite(total_lengths)) > 5: # only calc if 4 points go into defining line
            slope, intercept, _,_,_ = stats.linregress(np.log(dimensions[:-1]),np.log(total_lengths[:-1]))
            D[ss] = 1-slope

        # calculate sinuosity
        P[ss] = total_dist[-1]/total_lengths[-1]

    return D, P

df['fractal_dim'], df['sinuosity'] = zip(*df.apply(get_fractal_sinuosity_df, args = (20, 3, pix2mm, 2), axis=1))
print('Done calculating fractal dimension and sinuosity')
    
    


Done calculating fractal dimension and sinuosity


In [266]:
# save as feather for lme models in R

lens = [len(item) for item in df['fractal_dim']]
all_sections = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "light": np.repeat(df['light'].values, lens),
                        "fractal_dim" : np.concatenate(df['fractal_dim'].values), "sinuosity" : np.concatenate(df['sinuosity'].values)    })

colony_R = [col.split('20190')[-1][1:] for col in all_sections['colony'].values.tolist()]
# subs_string = all_sections['substrate'].values.tolist()
# substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
df_med_R = pd.DataFrame( { "colony" : colony_R, "substrate" : all_sections['substrate'], "trackway": all_sections['trackway'],
                       "light" : all_sections['light'], "fractal_dim": all_sections['fractal_dim'], "sinuosity": all_sections['sinuosity']} )
print('%i trackways in dataframe for analysis in R'%len(df_med_R))

# SAVE AS FEATHER FOR USE WITH R
print('\nSaving fractal dimension and sinuosity data as feather for use in R')
feather.write_dataframe(df_med_R, vid_locations + 'Fractal_Sinuosity_Data.feather')
print('Done saving')

6288 trackways in dataframe for analysis in R

Saving fractal dimension and sinuosity data as feather for use in R
Done saving


In [267]:
# plot fractal and sinuosity - subplots are substrates


lens = [len(item) for item in df['fractal_dim']]
all_sections = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "light": np.repeat(df['light'].values, lens),
                        "fractal_dim" : np.concatenate(df['fractal_dim'].values), "sinuosity" : np.concatenate(df['sinuosity'].values)    
                            })


plt.close('all')
plt.figure()
# my_pal = {"Flat": '#464F56', "Array": '#BA4246', "Step": '#087E8B', "C_Step": '#701C6F'}
my_pal = {0: '#464F56', 2: '#BA4246', 3: '#087E8B', 1: '#701C6F'}
for kk, subtype in enumerate(np.array(subtypes)[[-2,0]]):
    print(subtype)
    plt.subplot(1,2,kk+1)
    ax = sns.violinplot(x = 'light', y = 'fractal_dim',  
                data = all_sections[(all_sections['substrate']==subtype)], 
                        cut = 0 , palette=my_pal)
    plt.xlabel(subtype)
    plt.ylim([1, 1.2])

plt.figure()    
for kk, subtype in enumerate(np.array(subtypes)[[-2,0]]):
    print(subtype)
    plt.subplot(1,2,kk+1)
    ax = sns.violinplot(x = 'light', y = 'sinuosity',  
                data = all_sections[(all_sections['substrate']==subtype)], 
                        cut = 0 , palette=my_pal)
    plt.xlabel(subtype)
    plt.ylim([0.8, 13])

    

Flat
Array
Flat
Array


In [ ]:
# plot fractal and sinuosity - subplots are light/dark


lens = [len(item) for item in df['fractal_dim']]
all_sections = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "light": np.repeat(df['light'].values, lens),
                        "fractal_dim" : np.concatenate(df['fractal_dim'].values), "sinuosity" : np.concatenate(df['sinuosity'].values)    
                            })


plt.close('all')
plt.figure()
my_pal = {"Flat": '#464F56', "Array": '#BA4246', "Step": '#087E8B', "C_Step": '#701C6F'}
for kk, ltype in enumerate([1,0]):
    print('Lighting: %i'%ltype)
    plt.subplot(1,2,kk+1)
#     ax = sns.violinplot(x = 'substrate', y = 'fractal_dim',  
#                 data = all_sections[ (all_sections['light']==ltype) & np.isin(all_sections['substrate'].values, np.array(subtypes)[[0,2]]) ], 
#                         cut = 0 , palette=my_pal)
    ax = sns.boxplot(x = 'substrate', y = 'fractal_dim',  
                data = all_sections[ (all_sections['light']==ltype) & np.isin(all_sections['substrate'].values, np.array(subtypes)[[0,2]]) ], palette=my_pal)
    plt.xlabel(ltype)
    plt.ylim([1, 1.2])

plt.figure()    
for kk, ltype in enumerate([1,0]):
    print('Lighting: %i'%ltype)
    plt.subplot(1,2,kk+1)
#     ax = sns.violinplot(x = 'substrate', y = 'sinuosity',  
#                 data = all_sections[ (all_sections['light']==ltype) & np.isin(all_sections['substrate'].values, np.array(subtypes)[[0,2]]) ], 
#                         cut = 0 , palette=my_pal)
    ax = sns.boxplot(x = 'substrate', y = 'sinuosity',  
                data = all_sections[ (all_sections['light']==ltype) & np.isin(all_sections['substrate'].values, np.array(subtypes)[[0,2]]) ], palette=my_pal)
    plt.xlabel(ltype)
    plt.ylim([0.99, 2])

In [162]:
# plot weird sinuosity values
plt.close('all')
for tr in [1367,1839,2488,2537,3094]:
    plt.figure()
    plt.plot(df.loc[tr]['x'], df.loc[tr]['y'], 'k-', alpha = 0.5)
    plt.plot(df.loc[tr]['x'], df.loc[tr]['y'], 'k.')
    plt.title(df.loc[tr]['sinuosity'])
#     plt.axes('equal')
    plt.xlim([0,1000])
    plt.ylim([0,550])

In [ ]:
# test out on specific trials
for tr in np.arange(1431,1438):
    plt.close('all')

    x = np.array(df['x_final'][tr]) # 'x' has jumps from removing frames close to edge and when ant is stopped for long time
    y = np.array(df['y_final'][tr])
    frames = df['frames'][tr]
    idcs = np.logical_and(np.isfinite(x), np.isfinite(y))
    x=x[idcs]; y = y[idcs]; frames= frames[idcs]

    # plot trajectory
    ax1=plt.subplot(2,1,1)
    ax2 =plt.subplot(2,1,2)

    # split into contiguous sections
    sections = np.split(np.arange(0,len(x)), np.where(np.diff(frames)>1)[0]+1)

    # make a richardson plot for each section to calc fractal dimension and sinuosity
    cutoff = 2
    D = np.ones(len(sections))*np.nan
    P = np.ones(len(sections))*np.nan
    for ss,section in enumerate( sections):

        plt.sca(ax1)
        # plt.plot(df['x_kal'][tr],df['y_kal'][tr], '.b', alpha = 0.4)
        plt.plot(x[section],y[section], '-k', alpha = 0.3) # removes sections with ants stopped --> don't want
        plt.plot(x[section],y[section], '.k', alpha = 0.8, markersize= 3)
        plt.xlim([0,1000])
        plt.ylim([0,550])

        # dont' calculate things if fewer than 20 frames of data in section
        if len(section)<20:
            print('section is too short')
            continue
            
        # total dist
        total_dist = np.cumsum(np.insert(np.linalg.norm([np.diff(x[section]),np.diff(y[section])], axis = 0),0,0))  
        
        # don't calculate if total distance traveled in less than __ mm
        if total_dist[-1]<3*pix2mm:
            print('ant travel distance is too short')
            continue
        
        # for each step distance, find total length
        dimensions = np.append(np.arange(5, total_dist[-1]/2, np.ceil(total_dist[-1]/15)), total_dist[-1]-0.01)
#         dimensions = np.append(np.arange(5, total_dist[-1]/2, pix2mm), total_dist[-1]-0.01)
        total_lengths = np.ones(dimensions.shape)*np.nan
        for dd, dimen in enumerate(dimensions):
            errors = np.repeat(total_dist[:,np.newaxis], total_dist[-1]/dimen+1,axis=1
                              ) - np.repeat(np.arange(0,total_dist[-1], dimen)[np.newaxis,:], len(total_dist), axis =0)
            min_errors = np.min(np.abs(errors), axis =0)
            if np.any(min_errors > cutoff):
                print('step length does not have data within cutoff for all steps')
            step_locations = np.argmin(np.abs(errors), axis =0)
            total_lengths[dd] = np.sum(np.linalg.norm([np.diff(x[section][step_locations]),np.diff(y[section][step_locations])], axis = 0))

        plt.sca(ax2)
        plt.plot(np.log(dimensions[1:-1]), np.log(total_lengths[1:-1]), '.', color = ['r','b','g','c'][ss])
        plt.plot(np.log(dimensions[np.array([0,-1])]), np.log(total_lengths[np.array([0,-1])]), '*', color = ['r','b','g','c'][ss])
        
        # find slope of all points not bookends, only if at least 4 points --> calculate fractal dist 
        if np.sum(np.isfinite(total_lengths)) > 5:
            slope, intercept, _,_,_ = stats.linregress(np.log(dimensions[:-1]),np.log(total_lengths[:-1]))
            plt.plot(np.log(dimensions[np.array([0,-2])]), intercept + slope*np.log(dimensions[np.array([0,-2])]), '--k', alpha = 0.5)
            D[ss] = 1-slope
        else:
            print('not enough dimension points to calculate fractal dimension')
        
        # calc sinuosity
        P[ss] = total_dist[-1]/total_lengths[-1]
        print(total_lengths[-1])
        
        
        plt.text(np.log(total_dist[-1]/3*2), np.log(total_dist[-1]/3*2), 'D: %0.2f'%(D[ss]))
        plt.text(np.log(total_dist[-1]/3*2), np.log(total_dist[-1]/3*1), 'P: %0.2f'%(P[ss]))
        plt.ylim([0,8])
        plt.xlabel('log step length')
        plt.ylabel('log trajectory length (pix)')
            

    plt.pause(3)
    

In [210]:
# try to normalize distribution
from scipy import stats

plt.close('all')

for subtype in np.array(subtypes)[[0,2]]:
    for ltype in [1,0]:

        ys = all_sections[ (all_sections['light']==ltype) & (all_sections['substrate'].values==subtype) ]['fractal_dim'].values
        ys = ys[np.isfinite(ys)]
        ys_bc, _ = stats.boxcox(ys)

#         plt.boxplot(ys, positions = [0])
#         plt.boxplot(ys**(1/2), positions = [1])
#         print(stats.shapiro(ys))
#         print(stats.shapiro(ys**(1/3)))
#         print(stats.shapiro(np.log(ys)))
#         plt.boxplot(ys_bc, positions = [1.5])
#         plt.xlim(-0.5,2)


        plt.figure()
        ax1 = plt.subplot(2,1,1)
        stats.probplot(ys, dist = stats.norm, plot= ax1)
        plt.title('sub: %s, light %i'%(subtype, ltype))
        ax2 = plt.subplot(2,1,2)
        stats.probplot(ys_bc, dist = stats.norm, plot= ax2)
        plt.title('')
        

ys = all_sections['fractal_dim'].values
ys = ys[np.isfinite(ys)]
ys_bc, _ = stats.boxcox(ys)
plt.figure()
ax1 = plt.subplot(2,1,1)
stats.probplot(ys, dist = stats.norm, plot= ax1)
plt.title('sub: %s, light %i'%(subtype, ltype))
ax2 = plt.subplot(2,1,2)
stats.probplot(ys_bc, dist = stats.norm, plot= ax2)
plt.title('')

Text(0.5,1,'')

# Normalize velocity by distance to step to average over all trials

In [63]:
# for each trackways, find ave velocity for each distance bin away from crossing obstacle

def ave_speed_by_bins_df(df, var_OI, resolution):
    x_raw = ((df['x_final']-df['cross_step_x'])*df['travel_dir'])/pix2mm
    v_raw = np.append(df[var_OI], np.nan)
    keep = np.logical_and(np.isfinite(v_raw) ,np.isfinite(x_raw))
    vs = v_raw[keep];  xs = x_raw[keep]; 
    
#     bins = np.arange(-15,15+resolution,resolution)
    bins = np.append(np.insert(np.arange(-5,5+resolution,resolution),0,-15),15)
    bin_idcs = np.digitize(xs,bins)
    vs_norm = np.ones(bins.shape)*np.nan

    bvals, idx, counts = np.unique(bin_idcs, return_inverse=True, return_counts = True)
    sum_vs = np.bincount(idx, weights = vs)
    mean_vs = sum_vs / counts
    vs_norm[bvals] = mean_vs
    return vs_norm


resolution = 0.5
step_df['v_final_norm'] = step_df.apply(ave_speed_by_bins_df, args = ('v_forward', resolution,), axis=1)
step_df['vy_final_norm'] = step_df.apply(ave_speed_by_bins_df, args = ('v_lateral', resolution,), axis=1)

print('done finding ave velocity for each x distance from obstacle bin')


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done finding ave velocity for each x distance from obstacle bin


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
# plot speed vs. distance from obstacle

plt.close('all')
plt.figure()

stOI = -1 # all step ups or downs
all_vs={}
ax={}
    
for ss, subOI in enumerate(['C_Step','Step']):
    for li, liOI in enumerate([1,0]):
        

        dataOI = step_df.loc[(step_df['substrate']==subOI) & (step_df['light']==liOI) & (step_df['step_dir']==stOI)]

        ax[2*ss+li]=plt.subplot(2,2,2*li+ss+1)
        plt.axvline(x=0,color='k',alpha = 0.4)
        for index, row in dataOI.iterrows():
                plt.plot(((row['x_final']-row['cross_step_x'])*row['travel_dir'])/pix2mm,
                         np.append(row['v_forward'], np.nan),'-k', alpha = 0.02)
                
        
        all_vs[2*ss+li] = np.vstack(dataOI['v_final_norm'].values)[:,1:]
        all_vys = np.vstack(np.abs(dataOI['vy_final_norm'].values))[:,1:]
        ave = np.nanmedian(all_vs[2*ss+li],axis=0)
        bins = np.append(np.insert(np.arange(-5,5+resolution,resolution),0,-15),15)
        xs = np.diff(bins)/2+bins[:-1]
        plt.plot(xs, ave, '.', color = pltcolors[(2*ss+li)%2])
        plt.plot(xs, ave, color = pltcolors[(2*ss+li)%2], linestyle =['-',':'][(2*ss+li)//2])
        plt.plot(xs+0.2, np.nanmean(all_vys,axis=0), '.',color = pltcolors[3])
        plt.plot(xs+0.2, np.nanmean(all_vys,axis=0), '-',color = pltcolors[3])

        plt.plot([xs,xs], [ave-np.nanstd(all_vs[2*ss+li],axis=0), 
                           ave+np.nanstd(all_vs[2*ss+li],axis=0)], '-', alpha = 0.5, color = pltcolors[(2*ss+li)%2])
        plt.plot([xs+0.2,xs+0.2], [np.nanmean(all_vys,axis=0)-np.nanstd(all_vys,axis=0), 
                               np.nanmean(all_vys,axis=0)+np.nanstd(all_vys,axis=0)], '-', alpha = 0.5,color = pltcolors[3])
        
        plt.xlim([-15,15])
        plt.ylim([0,60])
        plt.ylabel('speed (mm/s)')
        plt.xlabel('distance to step')
        plt.text(8,53,'n: %i'%all_vs[2*ss+li].shape[0])
        if li==0:
            plt.gca().get_xaxis().set_visible(False)
            plt.title('Sub: %s'%subOI,loc='left')
        if ss==1:
            plt.gca().get_yaxis().set_visible(False)

            
# calculate sig differences among each velocity bin
step_vals = np.ones(len(bins)-1)*np.nan
cstep_vals = np.ones(len(bins)-1)*np.nan
light_vals = np.ones(len(bins)-1)*np.nan
dark_vals = np.ones(len(bins)-1)*np.nan

for b_OI in np.arange(len(bins)-1):
    _,cstep_vals[b_OI] = stats.ttest_ind(all_vs[0][:,b_OI], all_vs[1][:,b_OI], nan_policy = 'omit')
    _,step_vals[b_OI] = stats.ttest_ind(all_vs[2][:,b_OI], all_vs[3][:,b_OI], nan_policy = 'omit')
    _,light_vals[b_OI] = stats.ttest_ind(all_vs[0][:,b_OI], all_vs[2][:,b_OI], nan_policy = 'omit')
    _,dark_vals[b_OI] = stats.ttest_ind(all_vs[1][:,b_OI], all_vs[3][:,b_OI], nan_policy = 'omit')
# print(cstep_vals)
# print(step_vals)
# print(light_vals)
# print(dark_vals)

    
#     plt.close('all')
plt.figure()
dodge = 0.05
for pp in np.arange(4):
    ave = np.nanmean(all_vs[pp],axis=0)
    plt.plot(xs-2*dodge+pp*dodge, ave, '.', color = pltcolors[pp%2])
    plt.plot(xs-2*dodge+pp*dodge, ave, color = pltcolors[pp%2], linestyle = ['-','--'][pp//2])
#     plt.plot([xs-2*dodge+pp*dodge,xs-2*dodge+pp*dodge], [ave-np.nanstd(all_vs[pp],axis=0), 
#                            ave+np.nanstd(all_vs[pp],axis=0)],  alpha = 0.3, color = pltcolors[pp%2], linestyle = ['-',':'][pp//2])

plt.plot(xs[light_vals<0.05], 30*np.ones(np.sum(light_vals<0.05)), '*', alpha = 0.5, color = pltcolors[0])
plt.plot(xs[dark_vals<0.05], 31*np.ones(np.sum(dark_vals<0.05)), '*', alpha = 0.5, color = pltcolors[1])
plt.plot(xs[cstep_vals<0.05], 32*np.ones(np.sum(cstep_vals<0.05)), 'vk', alpha = 0.5)
plt.plot(xs[step_vals<0.05], 33*np.ones(np.sum(step_vals<0.05)), 'vk', alpha = 0.5, markerfacecolor='white')

In [68]:
# plot out step down velocity traces by colony to see what's going on

plt.close('all')
plt.figure(figsize=(7,12))
coltypes = sorted(list(set(df['colony'].values)))

stOI = -1 # all step ups or downs
liOI = 1 
# all_vs={}
# ax={}
    

for cc,colOI in enumerate(coltypes):
    for ss, subOI in enumerate(['C_Step','Step']):
        

        dataOI = step_df.loc[(step_df['substrate']==subOI) & (step_df['light']==liOI) 
                             & (step_df['step_dir']==stOI) & (step_df['colony']==colOI)]

        plt.subplot(len(coltypes),2,2*cc+ss+1)
        plt.axvline(x=0,color='k',alpha = 0.4)
        for index, row in dataOI.iterrows():
                plt.plot(((row['x_final']-row['cross_step_x'])*row['travel_dir'])/pix2mm,
                         np.append(row['v_forward'], np.nan),'-k', alpha = 0.2)
                
        
#         all_vs[2*ss+li] = np.vstack(dataOI['v_final_norm'].values)[:,1:]
#         all_vys = np.vstack(np.abs(dataOI['vy_final_norm'].values))[:,1:]
#         ave = np.nanmedian(all_vs[2*ss+li],axis=0)
#         bins = np.append(np.insert(np.arange(-5,5+resolution,resolution),0,-15),15)
#         xs = np.diff(bins)/2+bins[:-1]
#         plt.plot(xs, ave, '.', color = pltcolors[(2*ss+li)%2])
#         plt.plot(xs, ave, color = pltcolors[(2*ss+li)%2], linestyle =['-',':'][(2*ss+li)//2])
#         plt.plot(xs+0.2, np.nanmean(all_vys,axis=0), '.',color = pltcolors[3])
#         plt.plot(xs+0.2, np.nanmean(all_vys,axis=0), '-',color = pltcolors[3])

#         plt.plot([xs,xs], [ave-np.nanstd(all_vs[2*ss+li],axis=0), 
#                            ave+np.nanstd(all_vs[2*ss+li],axis=0)], '-', alpha = 0.5, color = pltcolors[(2*ss+li)%2])
#         plt.plot([xs+0.2,xs+0.2], [np.nanmean(all_vys,axis=0)-np.nanstd(all_vys,axis=0), 
#                                np.nanmean(all_vys,axis=0)+np.nanstd(all_vys,axis=0)], '-', alpha = 0.5,color = pltcolors[3])
        
        plt.xlim([-15,15])
        plt.ylim([0,60])
        plt.xlabel('distance to step')
        plt.ylabel(colOI[-4:])
        plt.text(8,53,'n: %i'%len(dataOI))
        if cc==0:
            plt.title('Sub: %s'%subOI,loc='left')
        if cc < len(coltypes)-1:
            plt.gca().get_xaxis().set_visible(False)
        if ss==1:
            plt.gca().get_yaxis().set_visible(False)


# X and Y velocities, forward and lateral velocities

In [17]:
# FORWARD AND LATERAL
x_min = -6
plt.close('all')

for tr in [4322]:# np.random.choice(step_df.loc[step_df['step_dir']==1].index, 5, replace=False):
    x_raw = ((step_df['x_final'][tr]-step_df['cross_step_x'][tr])*step_df['travel_dir'][tr])/pix2mm
    y_raw = step_df['y_final'][tr]/pix2mm
    
    if np.any(np.isfinite(x_raw)) & (np.nanmin(x_raw)<x_min) & (np.isfinite(step_df['light'][tr])) : # is there any data?
        
        print('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
              (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]))
        v_raw = step_df['v_final'][tr]/pix2mm
        vx_raw = np.append(step_df['v_forward'][tr], np.nan)
        vy_raw = np.append(step_df['v_lateral'][tr], np.nan)
        c_idx = int(step_df['cross_step_idx'][tr])
        c_x = step_df['cross_step_x'][tr]
        c_y = step_df['cross_step_y'][tr]
        ti = (np.arange(0,len(x_raw))-c_idx)/fps
        
        
        # is there a big block of nan preventing analysis?
        x_min_idx = np.nanargmin(1/(x_raw-x_min)) #np.where(np.abs(x_raw-x_min)<.2)[0][0] #np.nanargmin(np.abs(x_raw-x_min))
        if np.sum(np.isnan(x_raw[x_min_idx:c_idx])) > 10:
            print('     too many nan before gets to obstacle')
            continue
        
        
        fig = plt.figure(figsize = (9,12))
        ax1 = plt.subplot(4,1,1)
#         plt.plot(step_df['x_final'][tr]/pix2mm, step_df['y_final'][tr]/pix2mm, '.k', alpha = 0.2)
        plt.plot(step_df['x_final'][tr]/pix2mm, step_df['y_final'][tr]/pix2mm, '-k', alpha = 0.3)
        plt.plot(step_df['cross_step_x'][tr]/pix2mm+step_df['travel_dir'][tr]*7, 7.5, 'ok')
        plt.axvline(x=step_df['cross_step_x'][tr]/pix2mm,color = 'k', alpha = 1)
        plt.ylim([0,550/pix2mm])
        plt.xlim([0,1000/pix2mm])
        plt.gca().invert_yaxis()
        plt.title('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
              (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]), loc = 'left')
        # add near edge cut-off
        col = df['colony'][tr]
        sub = df['substrate'][tr]
        idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
        p = sub_edge_df.iloc[idx]['hlines'].values[0]   
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm, 'k')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-2, ':r')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-16, 'k')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-14, ':r')
        
        plt.subplot(4,1,2)
        plt.plot(x_raw, step_df['angle_smooth'][tr], '-k', alpha = 0.3)
        plt.plot(x_raw, step_df['angle_improved'][tr], '--k', alpha = 0.3)
        plt.plot(x_raw, step_df['angle'][tr]-90, ':k', alpha = 0.3)
        plt.axvline(x=0,color = 'k', alpha = 0.3)
        plt.ylim([-180,180])
        plt.ylabel('smoothed angle')
        
        plt.subplot(4,1,3)
        plt.plot(x_raw,vy_raw, '-r', alpha = 0.3)
        plt.plot(x_raw,vx_raw, '-b', alpha = 0.3)
        plt.plot(x_raw, lowpass_filt_sections(vy_raw, 0.06) , 'r', alpha = 0.5, linewidth = 2)
        plt.plot(x_raw, lowpass_filt_sections(vx_raw, 0.06), 'b', alpha = 0.5, linewidth = 2)
        plt.axvline(x=0,color = 'k', alpha = 0.3)
        plt.axhline(y=0,color = 'k', alpha = 0.3)     
        plt.ylim([-30,50])
        plt.xlabel('x dist from step')
        plt.ylabel('velocity (mm/s)')
        
        ax4=plt.subplot(4,1,4)
        plt.plot(step_df['frames'][tr],vy_raw, '-r', alpha = 0.3)
        plt.plot(step_df['frames'][tr],vx_raw, '-b', alpha = 0.3)
        plt.plot(step_df['frames'][tr], lowpass_filt_sections(vy_raw, 0.06) , 'r', alpha = 0.5, linewidth = 2)
        plt.plot(step_df['frames'][tr], lowpass_filt_sections(vx_raw, 0.06), 'b', alpha = 0.5, linewidth = 2)
        plt.axvline(x=step_df['frames'][tr][c_idx],color = 'k', alpha = 0.3)
        plt.axhline(y=0,color = 'k', alpha = 0.3)     
        plt.ylim([-30,50])
        plt.xlabel('frame')
        plt.ylabel('velocity (mm/s)')
        
        ax2 = ax4.twinx()
        ax2.plot(step_df['frames'][tr][:-1], np.diff(lowpass_filt_sections(vy_raw, 0.06))*fps,':r', alpha = 0.2)
        ax2.plot(step_df['frames'][tr][:-1], np.diff(lowpass_filt_sections(vx_raw, 0.06))*fps,':b', alpha = 0.2)
        plt.ylabel('accel (mm/s^2)')
        plt.axhline(y=0,color = 'k', alpha = 0.3, linestyle=':')
        plt.ylim([-300,300])



Tr: 4322 -- Sub: Step -- Light: 1 -- Step Direction: 1


In [137]:
# X AND Y
x_min = -6
plt.close('all')

for tr in np.random.choice(step_df.loc[step_df['step_dir']==1].index, 5, replace=False):
    x_raw = ((step_df['x_final'][tr]-step_df['cross_step_x'][tr])*step_df['travel_dir'][tr])/pix2mm
    y_raw = step_df['y_final'][tr]/pix2mm
    
    if np.any(np.isfinite(x_raw)) & (np.nanmin(x_raw)<x_min) & (np.isfinite(step_df['light'][tr])) : # is there any data?
        
        print('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
              (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]))
        v_raw = step_df['v_final'][tr]/pix2mm
        vx_raw = np.append(np.diff(x_raw)*fps, np.nan)
        vy_raw = np.append(np.diff(y_raw)*fps, np.nan)
        c_idx = int(step_df['cross_step_idx'][tr])
        c_x = step_df['cross_step_x'][tr]
        c_y = step_df['cross_step_y'][tr]
        ti = (np.arange(0,len(x_raw))-c_idx)/fps
        
        
        # is there a big block of nan preventing analysis?
        x_min_idx = np.nanargmin(1/(x_raw-x_min)) #np.where(np.abs(x_raw-x_min)<.2)[0][0] #np.nanargmin(np.abs(x_raw-x_min))
        if np.sum(np.isnan(x_raw[x_min_idx:c_idx])) > 10:
            print('     too many nan before gets to obstacle')
            continue
        
        
        fig,ax1 = plt.subplots(figsize = (12,5))
        plt.plot(x_raw,vy_raw, '-r', alpha = 0.3)
        plt.plot(x_raw,vx_raw, '-b', alpha = 0.3)
        plt.plot(x_raw, lowpass_filt_sections(vy_raw, 0.06) , 'r', alpha = 0.5, linewidth = 2)
        plt.plot(x_raw, lowpass_filt_sections(vx_raw, 0.06), 'b', alpha = 0.5, linewidth = 2)

        
        plt.axvline(x=0,color = 'k', alpha = 0.3)
        plt.axhline(y=0,color = 'k', alpha = 0.3)
        
        plt.ylim([-30,50])

        
        
        ax2 = ax1.twinx()
        ax2.plot(x_raw[:-1], np.diff(lowpass_filt_sections(vy_raw, 0.06))*fps,':r', alpha = 0.2)
        ax2.plot(x_raw[:-1], np.diff(lowpass_filt_sections(vx_raw, 0.06))*fps,':b', alpha = 0.2)
        plt.ylabel('accel (mm/s^2)')
        plt.axhline(y=0,color = 'k', alpha = 0.3, linestyle=':')
        plt.ylim([-300,300])


#         v_movavg = take_moving_ave(v_raw, 24, 5)
#         v_lowpass = lowpass_filt_sections(v_raw, 0.06)
# #         plt.plot(x_raw, v_movavg, '-r', alpha = 0.3)
#         plt.plot(x_raw, v_lowpass, '-g', alpha = 0.3)
#         plt.title('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
#               (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]))

Tr: 2757 -- Sub: C_Step -- Light: 1 -- Step Direction: 1
Tr: 1896 -- Sub: Step -- Light: 1 -- Step Direction: 1
Tr: 1959 -- Sub: Step -- Light: 1 -- Step Direction: 1
Tr: 478 -- Sub: C_Step -- Light: 0 -- Step Direction: 1
Tr: 2420 -- Sub: Step -- Light: 0 -- Step Direction: 1


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()


# Find distance and time from step up that ant decelerates

In [90]:
# try out on single trial -- average speed for distance bins around when cross obstacle
from scipy.signal import argrelextrema
def take_moving_ave(x, N, n_nan_cutoff):
    padded_x = np.insert(np.insert( np.insert(x, len(x), np.empty(int(N/2))*np.nan), 0, np.empty(int(N/2))*np.nan ),0,0)
    n_nan = np.cumsum(np.isnan(padded_x))
    cumsum = np.nancumsum(padded_x) 
    window_sum = cumsum[N+1:] - cumsum[:-(N+1)] #- x # subtract value of interest from sum of all values within window
    window_n_nan = n_nan[N+1:] - n_nan[:-(N+1)] #- np.isnan(x)
    window_n_values = (N - window_n_nan + 1)
    movavg = (window_sum) / (window_n_values)
    movavg[window_n_nan>n_nan_cutoff]=np.nan
    return movavg
def lowpass_filt_sections(arr, fr):
    yy = np.isnan(arr)
    xx = range(len(yy))
    full_filtered = np.empty(yy.shape)*np.nan
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == False: # if is a group of nan
            g = list(g)
#             print('section to lowpass fitler: ', len(g))
            if len(g)>9:
                b, a = signal.butter(2,fr,btype='low')
                new_arr = arr[np.array(g)]
                mirror_arr = np.hstack([np.flip(new_arr,axis=0), new_arr, np.flip(new_arr,axis=0)])
                filtered = signal.filtfilt(b, a, mirror_arr)
                full_filtered[np.array(g)]=filtered[len(new_arr):-1*len(new_arr)]
    return full_filtered



tr = 4863 #114#4813

# [521, 2376,688, 461, 2229, 2211, 3294]
# [ 231, 232, 234, 243]
x_min = -7
plt.close('all')
for tr in [485]:#np.random.choice(step_df.loc[step_df['step_dir']==1].index, 5, replace=False):
    x_raw = ((step_df['x_final'][tr]-step_df['cross_step_x'][tr])*step_df['travel_dir'][tr])/pix2mm
    y_raw = step_df['y_final'][tr]/pix2mm
    
    if np.any(np.isfinite(x_raw)) & (np.nanmin(x_raw)<x_min): # is there any data?
        
        print('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
              (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]))
        v_raw = step_df['v_final'][tr]/pix2mm
#         vx_raw = np.diff(x_raw)*fps
#         vy_raw = np.diff(y_raw)*fps
        vx_raw = np.append(step_df['v_forward'][tr], np.nan)
        vy_raw = np.append(step_df['v_lateral'][tr], np.nan)
        
        
        # does ant go both directions across substrate?
        tmp = np.diff(x_raw*pix2mm)
        bin_counts,_ = np.histogram(tmp[np.isfinite(tmp)],bins = [-5,-1,1,5])
#         print( np.abs( (bin_counts[2]+1)/(bin_counts[0]+1)) )
        if np.abs( (bin_counts[2]+1)/(bin_counts[0]+1))<10:
            print('ant walks in both forward and backward directions')
            continue
        
        
        
        # PART 1 ---- normalize x position so can average across all trials
        # only keep non-nan values to average across normalized x positions
        keep = np.logical_and(np.isfinite(v_raw) ,np.isfinite(x_raw))
        vs = v_raw[keep];  xs = x_raw[keep]; 

        resolution = 0.05
        bins = np.arange(-15,15+resolution,resolution)
        bin_idcs = np.digitize(xs,bins)
        vs_norm = np.ones(bins.shape)*np.nan
        bvals, idx, counts = np.unique(bin_idcs, return_inverse=True, return_counts = True)
        sum_vs = np.bincount(idx, weights = vs)
        mean_vs = sum_vs / counts
        vs_norm[bvals] = mean_vs


        # PART 2 ----  find time and distance of deceleration
        c_idx = int(step_df['cross_step_idx'][tr])
        c_x = step_df['cross_step_x'][tr]
        c_y = step_df['cross_step_y'][tr]
        v_movavg = take_moving_ave(vx_raw, 24, 5)
        v_lowpass = lowpass_filt_sections(vx_raw, 0.06)
        
        #         plt.figure(figsize = (12,5))
        fig = plt.figure(figsize = (9,12))
        ax1 = plt.subplot(4,1,1)
        plt.plot(step_df['x_final'][tr]/pix2mm, step_df['y_final'][tr]/pix2mm, '-k', alpha = 0.3)
        plt.plot(step_df['cross_step_x'][tr]/pix2mm+step_df['travel_dir'][tr]*7, 7.5, 'ok')
        plt.axvline(x=step_df['cross_step_x'][tr]/pix2mm,color = 'k', alpha = 1)
        plt.ylim([0,550/pix2mm])
        plt.xlim([0,1000/pix2mm])
        plt.gca().invert_yaxis()
        plt.title('Tr: %i -- Sub: %s -- Light: %i -- Step Direction: %0.0f'%
              (tr, step_df['substrate'][tr], step_df['light'][tr], step_df['step_dir'][tr]), loc = 'left')
        # add near edge cut-off
        col = df['colony'][tr]
        sub = df['substrate'][tr]
        idx = sub_edge_df[(sub_edge_df['colony']==col) & (sub_edge_df['substrate']==sub)].index
        p = sub_edge_df.iloc[idx]['hlines'].values[0]   
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm, 'k')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-2, ':r')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-16, 'k')
        plt.plot(np.arange(0,1000)/pix2mm, (p[0] + p[1]*np.arange(0,1000))/pix2mm-14, ':r')
        
        plt.subplot(4,1,2)
        plt.plot(x_raw, step_df['angle_smooth'][tr], '-k', alpha = 0.3)
        plt.plot(x_raw, step_df['angle_improved'][tr], '--k', alpha = 0.3)
        plt.plot(x_raw, step_df['angle'][tr]-90, ':k', alpha = 0.3)
        plt.axvline(x=0,color = 'k', alpha = 0.3)
        plt.ylim([-180,180])
        plt.xlim([-15,15])
        plt.ylabel('smoothed angle')
        
        ax3=plt.subplot(4,1,3)
        plt.plot(x_raw,vy_raw, '-r', alpha = 0.3)
        plt.plot(x_raw,vx_raw, '-b', alpha = 0.3)
        plt.plot(x_raw, lowpass_filt_sections(vy_raw, 0.06) , 'r', alpha = 0.5, linewidth = 2)
        plt.plot(x_raw, lowpass_filt_sections(vx_raw, 0.06), 'b', alpha = 0.5, linewidth = 2)
        plt.axvline(x=0,color = 'k', alpha = 0.3)
        plt.axhline(y=0,color = 'k', alpha = 0.3)     
        plt.ylim([-30,50])
        plt.xlim([-15,15])
        plt.xlabel('x dist from step')
        plt.ylabel('velocity (mm/s)')
        
        ax4=plt.subplot(4,1,4)
        plt.plot(step_df['frames'][tr],vy_raw, '-r', alpha = 0.3)
        plt.plot(step_df['frames'][tr],vx_raw, '-b', alpha = 0.3)
        plt.plot(step_df['frames'][tr], lowpass_filt_sections(vy_raw, 0.06) , 'r', alpha = 0.5, linewidth = 2)
        plt.plot(step_df['frames'][tr], lowpass_filt_sections(vx_raw, 0.06), 'b', alpha = 0.5, linewidth = 2)
        plt.axvline(x=step_df['frames'][tr][c_idx],color = 'k', alpha = 0.3)
        plt.axhline(y=0,color = 'k', alpha = 0.3)     
        plt.ylim([-30,50])
        plt.xlabel('frame')
        plt.ylabel('velocity (mm/s)')
        
        ax2 = ax4.twinx()
        ax2.plot(step_df['frames'][tr][:-1], np.diff(lowpass_filt_sections(vy_raw, 0.06))*fps,':r', alpha = 0.2)
        ax2.plot(step_df['frames'][tr][:-1], np.diff(lowpass_filt_sections(vx_raw, 0.06))*fps,':b', alpha = 0.2)
        plt.ylabel('accel (mm/s^2)')
        plt.axhline(y=0,color = 'k', alpha = 0.3, linestyle=':')
        plt.ylim([-300,300])
        
        
        # CALCULATE THINGS
        
    
        # if it's a step down, look for decel after crossing step too
        if step_df['step_dir'][tr] == -1:
            c_idx = c_idx + 20
    
        # is there a big block of nan preventing analysis?
        x_min_idx = np.nanargmin(1/(x_raw-x_min)) #np.where(np.abs(x_raw-x_min)<.2)[0][0] #np.nanargmin(np.abs(x_raw-x_min))
        if np.sum(np.isnan(x_raw[x_min_idx:c_idx])) >5:
            print('too many nan before gets to obstacle')
            plt.pause(2)
            continue

        # use acceleration to find when decelerates
        a_cutoff = -150
        a = np.diff(v_lowpass)*fps
        
        # does ant slow down at all
        if np.nanmin(a[x_min_idx:c_idx]) > -100:
            print('does not slow down')
            plt.pause(3)
            continue
        
        biggest_valley = np.nanargmin(a[x_min_idx:c_idx]) + x_min_idx
        valleys = argrelextrema(a[x_min_idx:(biggest_valley+4)], np.less, order = 2) + x_min_idx
        valleys = valleys[ np.logical_and( x_raw[valleys]>-5, a[valleys]<-100) ]
        if len(valleys)>1:
            peaks_in_between = [np.any(a[valleys[i]:valleys[i+1]]>-50) for i in np.arange(0,len(valleys)-1)]
            peak_OI = np.where(np.logical_not(np.append(peaks_in_between,False))) [0][0]
            d_i2 = valleys[peak_OI]
            
        elif len(valleys)==1:
            d_i2 = valleys[0]
        else:
            print('no single decel identified')
            continue
        
        print(c_idx, len(a))
    
        plt.sca(ax3)
        plt.plot(x_raw[d_i2-3], v_lowpass[d_i2-3], '+g')
        plt.axvline(x=x_raw[d_i2], color='g', alpha = 0.1, linestyle = ':')
        plt.axvline(x=x_raw[d_i2-3], color='g', alpha = 0.3, linestyle = '-')
        
        plt.sca(ax4)
        plt.plot(step_df['frames'][tr][d_i2-3], v_lowpass[d_i2-3], '+g')
        plt.axvline(x=step_df['frames'][tr][d_i2], color='g', alpha = 0.1, linestyle = ':')
        plt.axvline(x=step_df['frames'][tr][d_i2-3], color='g', alpha = 0.3, linestyle = '-')

        
        decel_x = x_raw[d_i2-3]
        decel_t = (int(step_df['cross_step_idx'][tr]) - d_i2 +3)/fps
        print('decel dist: %0.2f mm -- decel time: %0.2f s'%(decel_x, decel_t))

        
        plt.pause(1)


Tr: 485 -- Sub: C_Step -- Light: 1 -- Step Direction: 1
does not slow down


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [133]:
# APPLY TO WHOLE DATAFRAME
from scipy.signal import argrelextrema

def find_decel_df(x, pix2mm, x_min):
    
    decel_x = np.nan; decel_t = np.nan;

    x_raw = ((x['x_final']-x['cross_step_x'])*x['travel_dir'])/pix2mm
    if np.any(np.isfinite(x_raw)) & (np.nanmin(x_raw)<x_min): # is there any data?
        v_raw = x['v_final']/pix2mm
        vx_raw = np.append(x['v_forward'], np.nan)
        vy_raw = np.append(x['v_lateral'], np.nan)
        c_idx = int(x['cross_step_idx'])
        c_x = x['cross_step_x']
        c_y = x['cross_step_y']
        v_lowpass = lowpass_filt_sections(vx_raw, 0.06)
        
        # if it's a step down, look for decel after crossing step too
        if step_df['step_dir'][tr] == -1:
            c_idx = c_idx + 20
        
        # does ant go both directions across substrate?
        tmp = np.diff(x_raw*pix2mm)
        bin_counts,_ = np.histogram(tmp[np.isfinite(tmp)],bins = [-5,-1,1,5])
        if np.abs( (bin_counts[2]+1)/(bin_counts[0]+1))<10:
            return decel_x, decel_t
        
        # if it's a step down, look for decel after crossing step too
        if x['step_dir'] == -1:
            c_idx = c_idx + 20
    
        # is there a big block of nan preventing analysis?
        x_min_idx = np.nanargmin(1/(x_raw-x_min)) 
        if np.sum(np.isnan(x_raw[x_min_idx:c_idx])) >5:
            return decel_x, decel_t
        
        # use acceleration to find when decelerates
        a_cutoff = -150
        a = np.diff(v_lowpass)*fps
   
        # does ant slow down at all
        if np.nanmin(a[x_min_idx:c_idx]) > -100:
            return decel_x, decel_t
        
        biggest_valley = np.nanargmin(a[x_min_idx:c_idx]) + x_min_idx
        valleys = argrelextrema(a[x_min_idx:(biggest_valley+4)], np.less, order = 2) + x_min_idx
        valleys = valleys[ np.logical_and( x_raw[valleys]>-5, a[valleys]<-100) ]
        if len(valleys)>1:
            peaks_in_between = [np.any(a[valleys[i]:valleys[i+1]]>-50) for i in np.arange(0,len(valleys)-1)]
            peak_OI = np.where(np.logical_not(np.append(peaks_in_between,False))) [0][0]
            d_i2 = valleys[peak_OI]
            
        elif len(valleys)==1:
            d_i2 = valleys[0]
        else:
            return decel_x, decel_t
        
        decel_x = x_raw[d_i2-3]
        decel_t = (int(x['cross_step_idx']) - d_i2 +3)/fps
        
    return decel_x, decel_t


x_min = -7 #only use trials that has speed data for at least this distance from the obstacle
step_df['decel_x'], step_df['decel_t'] = zip(*step_df.apply(
    find_decel_df, args = (pix2mm, x_min), axis=1))

print('done saving deceleration distance and time info')

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: All-NaN slice encountered
  if __name__ == '__main__':
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/scipy/signal/_peak_finding.py:73: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, plus)
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/scipy/signal/_peak_finding.py:74: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, minus)


done saving deceleration distance and time info


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [138]:
# BOXPLOT OF DECEL DATA

step_dir = 1

plt.close('all')
plt.figure()
plt.subplot(1,2,1)
bp=sns.boxplot(x="light", y = "decel_x", hue = "substrate", data = step_df.loc[step_df['step_dir']==step_dir], order = [1,0])
plt.title('stepping up', loc = 'left')
# plt.title('stepping down', loc = 'left')
plt.ylabel('deceleration dist from step (mm)')
plt.ylim([0,-6])
print(step_df.groupby(['step_dir','light','substrate']).apply(lambda x: len(x)))

plt.subplot(1,2,2)
sns.boxplot(x="light", y = "decel_t", hue = "substrate", data = step_df.loc[step_df['step_dir']==step_dir], order = [1,0])
plt.ylabel('deceleration time from crossing step (s)')
plt.ylim([0,1.5])



# VIOLIN PLOT?

# plt.figure()
# plt.subplot(1,2,1)
# sns.violinplot(x="light", y = "decel_x", hue = "substrate", data = step_df.loc[step_df['step_dir']==step_dir], order = [1,0])
# plt.title('stepping down', loc = 'left')
# plt.ylabel('deceleration dist from step (mm)')


step_dir  light  substrate
-1.0      0.0    C_Step        76
                 Step          67
          1.0    C_Step       112
                 Step         100
 1.0      0.0    C_Step        61
                 Step          66
          1.0    C_Step        83
                 Step         130
dtype: int64


(0, 1.5)

## Plot out all walking trajectories

For each colony, plots out trajectories on each substrate. Good way of visualizing if there are pheromone paths etc.

In [18]:
# FIND PHEROMONE TRAILS BASED ON Y LOCATION
from ipynb.fs.defs.LoadInTrackedData import create_trial_info
trial_info = create_trial_info(file_list)

allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
allcols = [tr['colony'] for tr in trial_info]
coltypes = sorted(list(set(allcols)))
wid = 1000
hei = 550

for coltype in coltypes:
    plt.figure(figsize = (20,4))
    plt.clf()
    col_data = df.loc[df['colony'] == coltype]
    
    for kk, subtype in enumerate(subtypes):
        dataOI = col_data.loc[col_data['substrate'] == subtype]
        
        # define subplot
        spwid=0.23
#         ax=plt.axes([0.05+spwid*int(kk), 0, spwid-.01-.07, 1])
        
        gs = gridspec.GridSpec(1,4)
        gs.update(left = 0.05+spwid*int(kk), right = 0.05+spwid*(int(kk)+1), wspace = 0)
        
        # plot all traces
        for index, row in dataOI.iterrows():
            ax1 = plt.subplot(gs[0,:-1])
            ax1.plot(row['x_raw'], row['y_raw'], '-k', linewidth = 1, alpha = 0.1)
            ax1.set_xlim((0, wid))
            ax1.set_ylim((0, hei))
        ax1.set_aspect(1)
        ax1.invert_yaxis()
        if kk > 0:
            ax1.get_yaxis().set_visible(False)
#         n_long = len(longtracks.loc[longtracks['colony'] == coltype].loc[ longtracks['substrate'] == subtype ])
#         plt.title(('%s -- n: %s -- n > 50 fr: %s' % (subtype, len(dataOI), n_long)), horizontalalignment = 'left', x = 0)
        plt.title(('%s -- n: %s' % (subtype, len(dataOI))), horizontalalignment = 'left', x = 0)
        
#         # plot hist of all y values
#         ax2 = plt.subplot(gs[0,-1])
        
#         # plotting through pandas
# #         all_ys = pd.DataFrame( {"y_raw" : np.concatenate(dataOI['y_raw'].values)})
# #         ax2=plt.axes([0.05+spwid*int(kk)+(spwid-0.07), .32, .07, .36])
# #         all_ys['y_raw'].hist(orientation = 'horizontal', alpha = 0.5)

#         # ploting through matplotlib
#         all_ys = np.concatenate(dataOI['y_raw'].values)
#         hist, bins = np.histogram(all_ys, bins = 20)
#         barwidth = bins[1]-bins[0]
#         centers = (bins[:-1]+bins[1:])/2                          
#         ax2.barh(centers, hist, align = 'center', height = barwidth, alpha = 0.5)
#         ax2.set_ylim(0,hei)
#         ax2.set_ylim(ax2.get_ylim()[::-1])
#         ax2.set_aspect(ax2.get_xbound()[1]/wid*3)
#         ax2.get_yaxis().set_visible(False)
#         ax2.get_xaxis().set_visible(False)
#         ax2.set_frame_on(False)
   
#         # identify pheromone trails using mode
#         ax1.axhline(centers[np.argmax(hist)],color='r', linestyle = '--', alpha = 0.5)
#         ax2.axhline(centers[np.argmax(hist)],color='r', linestyle = '--', alpha = 0.5, xmax = 0.95)
#         ax1.axhspan(centers[np.argmax(hist)]-2*barwidth, centers[np.argmax(hist)]+2*barwidth, color = 'r', alpha = 0.2)
        
#         # identify pheromone trails using median
#         ax1.axhline(np.mean(all_ys), alpha = 0.5)
#         ax1.axhspan(np.mean(all_ys)-np.std(all_ys), np.mean(all_ys)+np.std(all_ys), alpha = 0.2)
        
        
        
        
    plt.suptitle('Colony: %s' % coltype, x=0.02, y = 0.93, fontsize = 16, horizontalalignment = 'left')
    
#     plt.savefig(vid_locations + 'Figures/' + 'PheromoneTrail_%s.png' % coltype)


*** BUILDING TRIAL LIST ***
    Done building trial list


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
# PLOT AND SAVE TRAJECTORIES FOR EACH HOUR OF RECORDING
import matplotlib.animation as animation


alldates = [tr['date'] for tr in trial_info]
alltimes = [tr['time'] for tr in trial_info]
dates = sorted(list(set(alldates)))

dpi = 300
nfig = 0

for date in dates:
    print('Date: ', date)
    whichones = [i for i,x in enumerate(alldates) if x == date]
    todaystimes = [alltimes[x] for x in whichones]
    todayshours = [int(x[0:2]) for x in todaystimes]
    
    
    for hour  in list(range(0,24)):
        whichoftoday = [whichones[i] for i,x in enumerate(todayshours) if 
                        (x == hour)]
        tracksubset = [tracks[i] for i in whichoftoday]
        trialnames = [file_list[i] for i in whichoftoday]
        
        fig = plt.figure()
        plt.clf()
        
        num_flights = 0
        cnt = 0
        for cnt, track in enumerate(tracksubset):
            if track != []:
                for k, obj in track.items():

                    if len(obj['x']) < 10:
                        continue

                    # plot with each trial as separate color IN PIXELS
                    plt.plot(obj['xfilt'], obj['yfilt'], '-')

                    # vary color based on velocity
        #             plt.set_cmap('copper')
        #             temp=obj['vfilt']/np.amax(obj['vfilt'])
        #             plt.scatter(pix2cm*obj['xfilt'][1::], pix2cm*obj['yfilt'][1::], #'-' ,
        #                      c=temp, edgecolor='none')#.tolist()).tolist()

                    num_flights += 1

#         print('Trials in hour ', hour, ': ', cnt) #whichoftoday)
        plt.xlabel('x (cm)')
        plt.ylabel('y (cm)')
        plt.title('%s -- %d:00-%d:00.\nn = %d' %(date, hour, hour+1,cnt) ,loc = 'left')
        plt.gca().set_aspect(1)
        plt.gca().invert_yaxis()
        plt.xlim((0, 1280))
        plt.ylim((0, 700))
        plt.gcf()
        
        # save figure
        pname = os.path.join(vid_locations, 'Trackway%d.png'%(nfig))
        plt.savefig(pname)
        plt.close('all')
        
        nfig = nfig +1
            
# save images as movie
os.system("ffmpeg -r 4 -i '" + vid_locations  + "Trackway%01d.png' -vcodec mpeg4 -y '" 
          + vid_locations + "Trackways.mp4'")

# delete all trackway vids
pics2delete = glob.glob(os.path.join(vid_locations, 'Trackway*.png'))
for pic in pics2delete:
    os.remove(pic)

In [19]:
# plot all trajectories for each substrate
plt.close('all')

wid = 1000
hei = 550

plt.figure(figsize = (20,4))
plt.clf()
    
for kk, subtype in enumerate(subtypes):

    dataOI =  df.loc[df['substrate'] == subtype]
    print('raw len:', len(dataOI))
    
    
    if subtype in ['Step', 'Gap']:
        crossing_center = dataOI['x_raw'].apply(lambda x: (np.min(x)<350) & (np.max(x)>650)).values
        dataOI = dataOI.loc[crossing_center]
        print(len(dataOI))

    # define subplot
    spwid=0.23
#         ax=plt.axes([0.05+spwid*int(kk), 0, spwid-.01-.07, 1])

    gs = gridspec.GridSpec(1,4)
    gs.update(left = 0.05+spwid*int(kk), right = 0.05+spwid*(int(kk)+1), wspace = 0)

    # plot all traces
    for index, row in dataOI.iterrows():
        ax1 = plt.subplot(gs[0,:-1])
        ax1.plot(row['x_raw'], row['y_raw'], '-k', linewidth = 1, alpha = 0.1)
        ax1.set_xlim((0, wid))
        ax1.set_ylim((0, hei))
    ax1.set_aspect(1)
    ax1.invert_yaxis()
    if kk > 0:
        ax1.get_yaxis().set_visible(False)
#         n_long = len(longtracks.loc[longtracks['colony'] == coltype].loc[ longtracks['substrate'] == subtype ])
#         plt.title(('%s -- n: %s -- n > 50 fr: %s' % (subtype, len(dataOI), n_long)), horizontalalignment = 'left', x = 0)
    plt.title(('%s -- n: %s' % (subtype, len(dataOI))), horizontalalignment = 'left', x = 0)


# plt.suptitle('Colony: %s' % coltype, x=0.02, y = 0.93, fontsize = 16, horizontalalignment = 'left')

# plt.savefig(vid_locations + 'Figures/' + 'PheromoneTrail_%s.png' % coltype)
# del dataOI

raw len 605


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


raw len 426
raw len 454
raw len 521
354


In [51]:
# plot only trials that cross center

plt.close('all')
plt.figure(figsize = (20,12))

wid = 1000
hei = 550

subtype = 'C_Step'

for kk, coltype in enumerate(coltypes):

    spwid=0.19
    gs = gridspec.GridSpec(3,3)
#     gs.update(left = 0.05+spwid*int(kk//3), right = 0.05+spwid*(int(kk//3)+1), wspace = 0)
    ax1 = plt.subplot(gs[kk//3,kk%3])

    dataOI =  df.loc[(df['substrate'] == subtype) & (df['colony'] == coltype)]
    crossing_center = dataOI['x_raw'].apply(lambda x: (np.min(x)<350) & (np.max(x)>650)).values
    dataOI = dataOI.loc[crossing_center]
    print(len(dataOI))
    for index, row in dataOI.iterrows():
#         ax1.plot(row['x'],row['v'],'-k', alpha = 0.2)
        ax1.plot(row['x_raw'],row['y_raw'],'-k', alpha = 0.2)
    ax1.set_xlim((0, wid))
#     ax1.set_ylim((0,2000))
    ax1.set_ylim((0,hei))
    plt.gca().invert_yaxis()
    plt.title(coltype)


25
81
32
79
66
62
64
106
38


In [43]:
for tr in np.arange(1200,1800):
    if np.sum(np.diff(df['frames'][tr])>1)>0:
        print(tr)